In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd 


In [2]:
df = pd.read_csv('./KICT_0115_DATA/dataset/location_and_label_rmNan.csv',encoding='CP949')
df

,No.,Location,ID,위험정도,일반영상,열화상영상,레이블영상,주소
0,1,wadong-dong,1,1,RGB0,THR0,LAB0,경기 파주시 와동동 1517
1,2,wadong-dong,1,1,RGB1,THR1,LAB1,경기 파주시 와동동 1517
2,3,wadong-dong,1,1,RGB2,THR2,LAB2,경기 파주시 와동동 1517
3,4,wadong-dong,1,1,RGB3,THR3,LAB3,경기 파주시 와동동 1517
4,5,wadong-dong,1,1,RGB4,THR4,LAB4,경기 파주시 와동동 1517
...,...,...,...,...,...,...,...,...
9995,9996,"seonghwa, gaesin, jungnim-dong",17,2,RGB9995,THR9995,LAB9995,충북 청주시 서원구 산남동 683
9996,9997,"seonghwa, gaesin, jungnim-dong",17,2,RGB9996,THR9996,LAB9996,충북 청주시 서원구 산남동 683
9997,9998,"seonghwa, gaesin, jungnim-dong",17,1,RGB9997,THR9997,LAB9997,충북 청주시 서원구 산남동 683
9998,9999,"seonghwa, gaesin, jungnim-dong",17,1,RGB9998,THR9998,LAB9998,충북 청주시 서원구 산남동 683


In [3]:
val = df.loc[:,["위험정도","레이블영상"]].values
danger,labPaths = val[:,0],val[:,1]
danger[:10],labPaths[:10]

(array([1, 1, 1, 1, 1, 1, 0, 2, 0, 0], dtype=object),
 array(['LAB0', 'LAB1', 'LAB2', 'LAB3', 'LAB4', 'LAB5', 'LAB6', 'LAB7',
        'LAB8', 'LAB9'], dtype=object))

In [4]:
dir = "targetData3"
rgbs = [os.path.join(dir,"R"+lab[3:]+'.png') for lab in labPaths]
thrs = [os.path.join(dir,"T"+lab[3:]+'.png') for lab in labPaths]
labs = [os.path.join(dir,lab+'.png') for lab in labPaths]
rgbs[:10],thrs[:10],labs[:10]

(['targetData3/R0.png',
  'targetData3/R1.png',
  'targetData3/R2.png',
  'targetData3/R3.png',
  'targetData3/R4.png',
  'targetData3/R5.png',
  'targetData3/R6.png',
  'targetData3/R7.png',
  'targetData3/R8.png',
  'targetData3/R9.png'],
 ['targetData3/T0.png',
  'targetData3/T1.png',
  'targetData3/T2.png',
  'targetData3/T3.png',
  'targetData3/T4.png',
  'targetData3/T5.png',
  'targetData3/T6.png',
  'targetData3/T7.png',
  'targetData3/T8.png',
  'targetData3/T9.png'],
 ['targetData3/LAB0.png',
  'targetData3/LAB1.png',
  'targetData3/LAB2.png',
  'targetData3/LAB3.png',
  'targetData3/LAB4.png',
  'targetData3/LAB5.png',
  'targetData3/LAB6.png',
  'targetData3/LAB7.png',
  'targetData3/LAB8.png',
  'targetData3/LAB9.png'])

In [5]:
i = 0
while i < len(rgbs):
    if not os.path.exists(rgbs[i]) or not os.path.exists(thrs[i]) or not os.path.exists(labs[i]):
        print()
        rgbs.pop(i)
        thrs.pop(i)
        labs.pop(i)
        danger = np.delete(danger,[i])
    i+=1

In [6]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset

from torchvision import transforms
from PIL import Image
import cv2 

imgtrans = transforms.Compose([
    transforms.Resize(size=(128,256)),
    transforms.ToTensor()
])

class CompleteRgbThrDataset(torch.utils.data.Dataset):
    def __init__(self,rgbList,thrList,labList,label,imgtrans = imgtrans):
        self.rgbList = rgbList
        self.thrList = thrList
        self.labList = labList
        self.imgtrans = imgtrans
        self.label = label 
        
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self,idx):
        rgb = Image.open(self.rgbList[idx]).convert('RGB')
        thr = Image.open(self.thrList[idx]).convert('RGB')
        # lab = Image.fromarray((np.array(Image.open(self.labList[idx]).convert('L'))>=127.0).astype(np.uint8)*255)
        lab = Image.open(self.labList[idx]).convert('L')
        label = torch.LongTensor([self.label[idx]])
        if self.imgtrans:
            rgb = self.imgtrans(rgb)
            thr = self.imgtrans(thr)
            lab = self.imgtrans(lab)
        lab = (lab>=0.5).type(torch.float32)
        
        return rgb,thr,lab,label 



In [7]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
import math



class Swish(nn.Module):

    def __init__(self, *args, **kwargs):
        super(Swish, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)


class ConvBNReLU(nn.Sequential):

    def __init__(self, in_planes, out_planes, kernel_size, stride=1, groups=1):
        padding = self._get_padding(kernel_size, stride)
        super(ConvBNReLU, self).__init__(
            nn.ZeroPad2d(padding),
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding=0, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes),
            Swish(),
        )

    def _get_padding(self, kernel_size, stride):
        p = max(kernel_size - stride, 0)
        return [p // 2, p - p // 2, p // 2, p - p // 2]

class SqueezeExcitation(nn.Module):

    def __init__(self, in_planes, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_planes, reduced_dim, 1),
            Swish(),
            nn.Conv2d(reduced_dim, in_planes, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)

class MBConvBlock(nn.Module):

    def __init__(self,
                 in_planes,
                 out_planes,
                 expand_ratio,
                 kernel_size,
                 stride,
                 reduction_ratio=4,
                 drop_connect_rate=0.2):
        super(MBConvBlock, self).__init__()
        self.drop_connect_rate = drop_connect_rate
        self.use_residual = in_planes == out_planes and stride == 1
        assert stride in [1, 2]
        assert kernel_size in [3, 5]

        hidden_dim = in_planes * expand_ratio
        reduced_dim = max(1, int(in_planes / reduction_ratio))

        layers = []
        # pw
        if in_planes != hidden_dim:
            layers += [ConvBNReLU(in_planes, hidden_dim, 1)]

        layers += [
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, kernel_size, stride=stride, groups=hidden_dim),
            # se
            SqueezeExcitation(hidden_dim, reduced_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, out_planes, 1, bias=False),
            nn.BatchNorm2d(out_planes),
        ]

        self.conv = nn.Sequential(*layers)

    def _drop_connect(self, x):
        if not self.training:
            return x
        keep_prob = 1.0 - self.drop_connect_rate
        batch_size = x.size(0)
        random_tensor = keep_prob
        random_tensor += torch.rand(batch_size, 1, 1, 1, device=x.device)
        binary_tensor = random_tensor.floor()
        return x.div(keep_prob) * binary_tensor

    def forward(self, x):
        if self.use_residual:
            return x + self._drop_connect(self.conv(x))
        else:
            return self.conv(x)
        


class ResConvBlock(torch.nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.layer = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
            torch.nn.Conv2d(in_channels=out_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
        ])
        self.shortcut = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                           out_channels=out_channels,
                           kernel_size=(1,1),stride=1),
            torch.nn.BatchNorm2d(out_channels)
        ])
    def forward(self,x):
        shortcut = self.shortcut(x)
        x = self.layer(x)
        return x + shortcut

    
class FusionBlock(torch.nn.Module):
    def __init__(self,channel,height,width):
        super(FusionBlock, self).__init__()
        self.weightT = torch.nn.Parameter(torch.rand(size=(channel,height,width),requires_grad = True))
        self.weightR = torch.nn.Parameter(torch.rand(size=(channel,height,width),requires_grad = True))
        
    def forward(self,thr,rgb):
        thrAlpha = torch.sigmoid(self.weightT)
        thrBeta = 1-thrAlpha
        
        rgbAlpha = torch.sigmoid(self.weightR)
        rgbBeta = 1-rgbAlpha
        
        thr_ = thrAlpha*thr + thrBeta*rgb
        rgb_ = rgbAlpha*rgb + rgbBeta*thr
        
        return thr_,rgb_

class MBConvIntertwinedUNet(torch.nn.Module):
    def __init__(self, n_class):
        super().__init__()
        self.n_features = [64,256,256,256,256]
        self.downsample = torch.nn.MaxPool2d(2)
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        

        self.rgbStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        self.thrStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        

        self.rgbLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])
        
        self.thrLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])
        self.thrLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        
        self.rgbLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])

        self.bridge = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[4]*2,out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1)

            ])
        
        self.fuse1 = FusionBlock(channel=self.n_features[0],height=128//2,width=256//2)
        self.fuse2 = FusionBlock(channel=self.n_features[1],height=128//4,width=256//4)
        self.fuse3 = FusionBlock(channel=self.n_features[2],height=128//8,width=256//8)
        self.fuse4 = FusionBlock(channel=self.n_features[3],height=128//16,width=256//16)
        
        
        self.dLayer4 = ResConvBlock(self.n_features[4]+self.n_features[3]*2,self.n_features[3])
        self.dLayer3 = ResConvBlock(self.n_features[3]+self.n_features[2]*2,self.n_features[2])
        self.dLayer2 = ResConvBlock(self.n_features[2]+self.n_features[1]*2,self.n_features[1])
        self.dLayer1 = ResConvBlock(self.n_features[1]+self.n_features[0]*2,self.n_features[0])
        
        self.out = torch.nn.Conv2d(self.n_features[0],n_class,1)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                fan_out = m.weight.size(0)
                init_range = 1.0 / math.sqrt(fan_out)
                nn.init.uniform_(m.weight, -init_range, init_range)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        
    def forward(self, rgb, thr):
        thrStem = self.thrStem(thr)
        rgbStem = self.rgbStem(rgb)  # 256 
        
        
        thr = self.downsample(thrStem)
        rgb = self.downsample(rgbStem) # 128
        thr,rgb = self.fuse1(thr,rgb)
        
        thrLayer1 = self.thrLayer1(thr)
        rgbLayer1 = self.rgbLayer1(rgb)
        
        
        thr = self.downsample(thrLayer1)
        rgb = self.downsample(rgbLayer1) # 64 
        thr,rgb = self.fuse2(thr,rgb)
        
        thrLayer2 = self.thrLayer2(thr)
        rgbLayer2 = self.rgbLayer2(rgb)
        
        
        thr = self.downsample(thrLayer2)
        rgb = self.downsample(rgbLayer2) # 32 
        thr,rgb = self.fuse3(thr,rgb)
        
        thrLayer3 = self.thrLayer3(thr)
        rgbLayer3 = self.rgbLayer3(rgb)
        
        
        thr = self.downsample(thrLayer3)
        rgb = self.downsample(rgbLayer3) # 16
        thr,rgb = self.fuse4(thr,rgb)
        
        thrLayer4 = self.thrLayer4(thr)
        rgbLayer4 = self.rgbLayer4(rgb)

        sumLayer4 = self.bridge(torch.cat([rgbLayer4,thrLayer4],dim=1))
        
        
        x = self.upsample(sumLayer4)
        x = torch.cat([x,rgbLayer3,thrLayer3],dim=1)
        x = self.dLayer4(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer2,thrLayer2],dim=1)
        x = self.dLayer3(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer1,thrLayer1],dim=1)
        x = self.dLayer2(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbStem,thrStem],dim=1)
        x = self.dLayer1(x)
        
        out = self.out(x)
        
        return out
        
model = MBConvIntertwinedUNet(n_class=1)



In [7]:
from sklearn.model_selection import train_test_split

rgbTrain,rgbTest,thrTrain,thrTest,labTrain,labTest,danTrain,danTest = train_test_split(rgbs,thrs,labs,danger,
        test_size = 0.2,random_state = 42)
rgbTrain,rgbValid,thrTrain,thrValid,labTrain,labValid,danTrain,danValid = train_test_split(rgbTrain,thrTrain,labTrain,danTrain,
        test_size = 0.2,random_state = 42)
len(rgbTrain),len(rgbValid),len(rgbTest),len(thrTrain),len(thrValid),len(thrTest),len(labTrain),len(labValid),len(labTest),len(danTrain),len(danValid),len(danTest)

(6398, 1600, 2000, 6398, 1600, 2000, 6398, 1600, 2000, 6398, 1600, 2000)

In [8]:
trainDataset = CompleteRgbThrDataset(rgbTrain,thrTrain,labTrain,danTrain)
validDataset = CompleteRgbThrDataset(rgbValid,thrValid,labValid,danValid)
testDataset = CompleteRgbThrDataset(rgbTest,thrTest,labTest,danTest)

batch_size = 4


trainLoader = torch.utils.data.DataLoader(
trainDataset,batch_size=batch_size,shuffle=True,num_workers=8)

validLoader = torch.utils.data.DataLoader(
validDataset,batch_size=batch_size,shuffle=True,num_workers=8)

testLoader = torch.utils.data.DataLoader(
testDataset,batch_size=batch_size,shuffle=True,num_workers=8)
len(trainLoader),len(validLoader),len(testLoader)

(1600, 400, 500)

In [10]:
image_datasets = {
    'train': trainDataset, 'val': validDataset
}

dataloaders = {
    'train': trainLoader,
    'val': validLoader
}

dataset_sizes = {
    x: len(image_datasets[x]) for x in image_datasets.keys()
}

dataset_sizes


from collections import defaultdict
import torch.nn.functional as F
# from loss import dice_loss


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def iou_score(output, target):
    smooth = 1e-5

    if torch.is_tensor(output):
        output = torch.sigmoid(output).data.cpu().numpy()
    if torch.is_tensor(target):
        target = target.data.cpu().numpy()
    output_ = output > 0.5
    target_ = target > 0.5
    intersection = (output_ & target_).sum()
    union = (output_ | target_).sum()

    return (intersection + smooth) / (union + smooth)


def dice_coef(output, target):
    smooth = 1e-5

    output = torch.sigmoid(output).view(-1).data.cpu().numpy()
    target = target.view(-1).data.cpu().numpy()
    intersection = (output * target).sum()

    return (2. * intersection + smooth) / \
        (output.sum() + target.sum() + smooth)


def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()    

    intersection = (pred * target).sum(dim=2).sum(dim=2)
    
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    
    return loss.mean()


def calc_loss(pred, target, metrics, bce_weight=0.5):   
    bce = F.binary_cross_entropy_with_logits(pred, target)
        
    pred = F.sigmoid(pred)
    dice = dice_loss(pred, target)
    
    loss = bce * bce_weight + dice * (1 - bce_weight)
    
    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)

    
    return loss


def print_metrics(metrics, epoch_samples, phase = 'test'):    
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
        
    print("{}: {}".format(phase, ", ".join(outputs)))    


def test_model(model,loader):
    model.eval()
    
    metrics = defaultdict(float)
    epoch_samples = 0
    iouMeter = AverageMeter()
    diceMeter = AverageMeter()
    with torch.no_grad():
        for rgb,thr,labels,danger in loader:
            rgb = rgb.to(device)
            thr = thr.to(device)
            labels = labels.to(device)             

            outputs = model(rgb,thr)

            loss = calc_loss(outputs, labels, metrics)
            epoch_samples += rgb.size(0)

            iou = iou_score(outputs,labels)
            dice = dice_coef(outputs,labels)
            iouMeter.update(iou, rgb.size(0))
            diceMeter.update(dice, rgb.size(0))

        print_metrics(metrics, epoch_samples,'test')
        print('test_model ##','iou:',iouMeter.avg,'dice:',diceMeter.avg)
        epoch_loss = metrics['loss'] / epoch_samples

    
    
def train_model(model, optimizer, scheduler, num_epochs=25,early_stop=False):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    
    history = {}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        since = time.time()
        
        iouMeter = AverageMeter()
        diceMeter = AverageMeter()
        valid_iouMeter = AverageMeter()
        valid_diceMeter = AverageMeter()
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])
                    
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0
            
            for rgb,thr, labels,danger in dataloaders[phase]:
                
                rgb = rgb.to(device)
                thr = thr.to(device)
                labels = labels.to(device)             

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(rgb,thr)
                    loss = calc_loss(outputs, labels, metrics)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += rgb.size(0)
                
                if phase == 'train':
                    iou = iou_score(outputs,labels)
                    dice = dice_coef(outputs,labels)
                    iouMeter.update(iou, rgb.size(0))
                    diceMeter.update(dice, rgb.size(0))
                    
                if phase == 'val':
                    iou = iou_score(outputs,labels)
                    dice = dice_coef(outputs,labels)
                    valid_iouMeter.update(iou, rgb.size(0))
                    valid_diceMeter.update(dice, rgb.size(0))

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples
            
            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print('train iou:',iouMeter.avg,'dice:',diceMeter.avg)
        print('valid iou:',valid_iouMeter.avg,'dice:',valid_diceMeter.avg)

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    if early_stop:
        model.load_state_dict(best_model_wts)
    return model

In [46]:
# 100 에폭 조기종료안함 bigger 모델 (256 256 256 256, decoder swish act 모델)

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_class = 1

model = MBConvIntertwinedUNet(n_class=1).to(device)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=25, gamma=0.1)

model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=100)

cuda:0
Epoch 0/99
----------
LR 0.0001


/compuworks/anaconda3/envs/py36torch/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/compuworks/anaconda3/envs/py36torch/lib/python3.6/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train: bce: 0.477029, dice: 0.351579, loss: 0.414304
val: bce: 0.314001, dice: 0.318031, loss: 0.316016
saving best model
train iou: 0.6795970937866311 dice: 0.7578095843757497
valid iou: 0.7534238384299492 dice: 0.8155385094873308
11m 57s
Epoch 1/99
----------
LR 0.0001
train: bce: 0.343421, dice: 0.326631, loss: 0.335026
val: bce: 0.309657, dice: 0.317851, loss: 0.313754
saving best model
train iou: 0.7368112368094704 dice: 0.7941947474860974
valid iou: 0.7608983912234956 dice: 0.8197611846081483
11m 58s
Epoch 2/99
----------
LR 0.0001
train: bce: 0.315539, dice: 0.318810, loss: 0.317175
val: bce: 0.281522, dice: 0.316241, loss: 0.298882
saving best model
train iou: 0.7531144583545522 dice: 0.8086845971946043
valid iou: 0.7727258247494774 dice: 0.8358932719042272
12m 29s
Epoch 3/99
----------
LR 0.0001
train: bce: 0.298453, dice: 0.313487, loss: 0.305970
val: bce: 0.256430, dice: 0.305977, loss: 0.281204
saving best model
train iou: 0.7649274619177515 dice: 0.8171796948495514
valid i

train: bce: 0.100891, dice: 0.073715, loss: 0.087303
val: bce: 0.406210, dice: 0.145016, loss: 0.275613
train iou: 0.9154959025098418 dice: 0.9411414671864632
valid iou: 0.827854147468155 dice: 0.8878178069412371
10m 55s
Epoch 33/99
----------
LR 1e-05
train: bce: 0.096446, dice: 0.071368, loss: 0.083907
val: bce: 0.414790, dice: 0.144529, loss: 0.279660
train iou: 0.9162664565121923 dice: 0.9418962856334236
valid iou: 0.8292781587207607 dice: 0.8872984378752575
10m 52s
Epoch 34/99
----------
LR 1e-05
train: bce: 0.094944, dice: 0.069616, loss: 0.082280
val: bce: 0.441455, dice: 0.144713, loss: 0.293084
train iou: 0.9171886579822242 dice: 0.9423960552445801
valid iou: 0.8289026858834575 dice: 0.8909960950017818
10m 52s
Epoch 35/99
----------
LR 1e-05
train: bce: 0.093434, dice: 0.067305, loss: 0.080370
val: bce: 0.412605, dice: 0.145185, loss: 0.278895
train iou: 0.9185784886227313 dice: 0.9432485915577812
valid iou: 0.8287374986745675 dice: 0.8911917922588524
10m 56s
Epoch 36/99
-----

train: bce: 0.068167, dice: 0.047583, loss: 0.057875
val: bce: 0.463826, dice: 0.138399, loss: 0.301112
train iou: 0.938190030697417 dice: 0.9576258043017427
valid iou: 0.8322138468005104 dice: 0.8935995396772494
10m 51s
Epoch 65/99
----------
LR 1.0000000000000002e-06
train: bce: 0.068901, dice: 0.046966, loss: 0.057934
val: bce: 0.453464, dice: 0.137464, loss: 0.295464
train iou: 0.9371440979393337 dice: 0.9570769730250205
valid iou: 0.8316329744580325 dice: 0.8921108120552336
10m 50s
Epoch 66/99
----------
LR 1.0000000000000002e-06
train: bce: 0.068613, dice: 0.045834, loss: 0.057223
val: bce: 0.467207, dice: 0.136726, loss: 0.301967
train iou: 0.93872136038961 dice: 0.9579653581009681
valid iou: 0.8323421272542092 dice: 0.8949583576614117
10m 48s
Epoch 67/99
----------
LR 1.0000000000000002e-06
train: bce: 0.068081, dice: 0.048046, loss: 0.058063
val: bce: 0.493763, dice: 0.137907, loss: 0.315835
train iou: 0.9385829471303432 dice: 0.9581557855798783
valid iou: 0.8319688941469103 d

train: bce: 0.067689, dice: 0.046175, loss: 0.056932
val: bce: 0.456184, dice: 0.136936, loss: 0.296560
train iou: 0.9379097152294629 dice: 0.957421692823233
valid iou: 0.8303604911819312 dice: 0.8954619986520523
10m 53s
Epoch 96/99
----------
LR 1.0000000000000002e-07
train: bce: 0.065194, dice: 0.045070, loss: 0.055132
val: bce: 0.572839, dice: 0.139049, loss: 0.355944
train iou: 0.9407196554407838 dice: 0.9599398204766784
valid iou: 0.8293604004533661 dice: 0.8935260972387984
10m 49s
Epoch 97/99
----------
LR 1.0000000000000002e-07
train: bce: 0.068300, dice: 0.045226, loss: 0.056763
val: bce: 0.464761, dice: 0.136581, loss: 0.300671
train iou: 0.9385529391859181 dice: 0.9585616424090336
valid iou: 0.832883577186086 dice: 0.8957477826991256
10m 50s
Epoch 98/99
----------
LR 1.0000000000000002e-07
train: bce: 0.064250, dice: 0.045508, loss: 0.054879
val: bce: 0.446893, dice: 0.139368, loss: 0.293131
train iou: 0.9410354280981065 dice: 0.9599841315648568
valid iou: 0.8298509788838361 

In [48]:
torch.cuda.empty_cache()

test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)

test: bce: 0.449022, dice: 0.144526, loss: 0.296774
test_model ## iou: 0.8224804950143241 dice: 0.8860352553615529
test: bce: 0.449022, dice: 0.144526, loss: 0.296774
test_model ## iou: 0.8266757576764523 dice: 0.8925320256333048
test: bce: 0.449022, dice: 0.144526, loss: 0.296774
test_model ## iou: 0.8245538907769636 dice: 0.8903669923189228
test: bce: 0.449022, dice: 0.144526, loss: 0.296774
test_model ## iou: 0.8240662715474829 dice: 0.890193010607604
test: bce: 0.449022, dice: 0.144526, loss: 0.296774
test_model ## iou: 0.8217954387171416 dice: 0.8875355724972548


In [86]:
torch.sigmoid(torch.rand(size=[1]))

tensor([0.5333])

pixel wise weight이 아니라 int 하나로 조정하는 모델

In [87]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
import math



class Swish(nn.Module):

    def __init__(self, *args, **kwargs):
        super(Swish, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)


class ConvBNReLU(nn.Sequential):

    def __init__(self, in_planes, out_planes, kernel_size, stride=1, groups=1):
        padding = self._get_padding(kernel_size, stride)
        super(ConvBNReLU, self).__init__(
            nn.ZeroPad2d(padding),
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding=0, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes),
            Swish(),
        )

    def _get_padding(self, kernel_size, stride):
        p = max(kernel_size - stride, 0)
        return [p // 2, p - p // 2, p // 2, p - p // 2]

class SqueezeExcitation(nn.Module):

    def __init__(self, in_planes, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_planes, reduced_dim, 1),
            Swish(),
            nn.Conv2d(reduced_dim, in_planes, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)

class MBConvBlock(nn.Module):

    def __init__(self,
                 in_planes,
                 out_planes,
                 expand_ratio,
                 kernel_size,
                 stride,
                 reduction_ratio=4,
                 drop_connect_rate=0.2):
        super(MBConvBlock, self).__init__()
        self.drop_connect_rate = drop_connect_rate
        self.use_residual = in_planes == out_planes and stride == 1
        assert stride in [1, 2]
        assert kernel_size in [3, 5]

        hidden_dim = in_planes * expand_ratio
        reduced_dim = max(1, int(in_planes / reduction_ratio))

        layers = []
        # pw
        if in_planes != hidden_dim:
            layers += [ConvBNReLU(in_planes, hidden_dim, 1)]

        layers += [
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, kernel_size, stride=stride, groups=hidden_dim),
            # se
            SqueezeExcitation(hidden_dim, reduced_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, out_planes, 1, bias=False),
            nn.BatchNorm2d(out_planes),
        ]

        self.conv = nn.Sequential(*layers)

    def _drop_connect(self, x):
        if not self.training:
            return x
        keep_prob = 1.0 - self.drop_connect_rate
        batch_size = x.size(0)
        random_tensor = keep_prob
        random_tensor += torch.rand(batch_size, 1, 1, 1, device=x.device)
        binary_tensor = random_tensor.floor()
        return x.div(keep_prob) * binary_tensor

    def forward(self, x):
        if self.use_residual:
            return x + self._drop_connect(self.conv(x))
        else:
            return self.conv(x)
        


class ResConvBlock(torch.nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.layer = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
            torch.nn.Conv2d(in_channels=out_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
        ])
        self.shortcut = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                           out_channels=out_channels,
                           kernel_size=(1,1),stride=1),
            torch.nn.BatchNorm2d(out_channels)
        ])
    def forward(self,x):
        shortcut = self.shortcut(x)
        x = self.layer(x)
        return x + shortcut

    
class FusionBlock(torch.nn.Module):
    def __init__(self,channel,height,width):
        super(FusionBlock, self).__init__()
        self.weightT = torch.nn.Parameter(torch.rand(size=(1,),requires_grad = True))
        self.weightR = torch.nn.Parameter(torch.rand(size=(1,),requires_grad = True))
        
    def forward(self,thr,rgb):
        thrAlpha = torch.sigmoid(self.weightT)
        thrBeta = 1-thrAlpha
        
        rgbAlpha = torch.sigmoid(self.weightR)
        rgbBeta = 1-rgbAlpha
        
        thr_ = thrAlpha*thr + thrBeta*rgb
        rgb_ = rgbAlpha*rgb + rgbBeta*thr
        
        return thr_,rgb_

class MBConvIntertwinedUNet(torch.nn.Module):
    def __init__(self, n_class):
        super().__init__()
        self.n_features = [64,256,256,256,256]
        self.downsample = torch.nn.MaxPool2d(2)
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        

        self.rgbStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        self.thrStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        

        self.rgbLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])
        
        self.thrLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])
        self.thrLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        
        self.rgbLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])

        self.bridge = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[4]*2,out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1)

            ])
        
        self.fuse1 = FusionBlock(channel=self.n_features[0],height=128//2,width=256//2)
        self.fuse2 = FusionBlock(channel=self.n_features[1],height=128//4,width=256//4)
        self.fuse3 = FusionBlock(channel=self.n_features[2],height=128//8,width=256//8)
        self.fuse4 = FusionBlock(channel=self.n_features[3],height=128//16,width=256//16)
        
        
        self.dLayer4 = ResConvBlock(self.n_features[4]+self.n_features[3]*2,self.n_features[3])
        self.dLayer3 = ResConvBlock(self.n_features[3]+self.n_features[2]*2,self.n_features[2])
        self.dLayer2 = ResConvBlock(self.n_features[2]+self.n_features[1]*2,self.n_features[1])
        self.dLayer1 = ResConvBlock(self.n_features[1]+self.n_features[0]*2,self.n_features[0])
        
        self.out = torch.nn.Conv2d(self.n_features[0],n_class,1)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                fan_out = m.weight.size(0)
                init_range = 1.0 / math.sqrt(fan_out)
                nn.init.uniform_(m.weight, -init_range, init_range)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        
    def forward(self, rgb, thr):
        thrStem = self.thrStem(thr)
        rgbStem = self.rgbStem(rgb)  # 256 
        
        
        thr = self.downsample(thrStem)
        rgb = self.downsample(rgbStem) # 128
        thr,rgb = self.fuse1(thr,rgb)
        
        thrLayer1 = self.thrLayer1(thr)
        rgbLayer1 = self.rgbLayer1(rgb)
        
        
        thr = self.downsample(thrLayer1)
        rgb = self.downsample(rgbLayer1) # 64 
        thr,rgb = self.fuse2(thr,rgb)
        
        thrLayer2 = self.thrLayer2(thr)
        rgbLayer2 = self.rgbLayer2(rgb)
        
        
        thr = self.downsample(thrLayer2)
        rgb = self.downsample(rgbLayer2) # 32 
        thr,rgb = self.fuse3(thr,rgb)
        
        thrLayer3 = self.thrLayer3(thr)
        rgbLayer3 = self.rgbLayer3(rgb)
        
        
        thr = self.downsample(thrLayer3)
        rgb = self.downsample(rgbLayer3) # 16
        thr,rgb = self.fuse4(thr,rgb)
        
        thrLayer4 = self.thrLayer4(thr)
        rgbLayer4 = self.rgbLayer4(rgb)

        sumLayer4 = self.bridge(torch.cat([rgbLayer4,thrLayer4],dim=1))
        
        
        x = self.upsample(sumLayer4)
        x = torch.cat([x,rgbLayer3,thrLayer3],dim=1)
        x = self.dLayer4(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer2,thrLayer2],dim=1)
        x = self.dLayer3(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer1,thrLayer1],dim=1)
        x = self.dLayer2(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbStem,thrStem],dim=1)
        x = self.dLayer1(x)
        
        out = self.out(x)
        
        return out
        
model = MBConvIntertwinedUNet(n_class=1)



In [88]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_class = 1

model = MBConvIntertwinedUNet(n_class=1).to(device)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=25, gamma=0.1)

model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=100)

cuda:0
Epoch 0/99
----------
LR 0.0001
train: bce: 0.494324, dice: 0.352371, loss: 0.423347
val: bce: 0.294692, dice: 0.329443, loss: 0.312067
saving best model
train iou: 0.6800349891281745 dice: 0.7589928908471285
valid iou: 0.7547379277496343 dice: 0.8041984553715466
10m 38s
Epoch 1/99
----------
LR 0.0001
train: bce: 0.342255, dice: 0.326754, loss: 0.334504
val: bce: 0.278434, dice: 0.305199, loss: 0.291817
saving best model
train iou: 0.736508299515843 dice: 0.7950128489215063
valid iou: 0.7836457426681891 dice: 0.8397390812556363
10m 51s
Epoch 2/99
----------
LR 0.0001
train: bce: 0.312392, dice: 0.317823, loss: 0.315108
val: bce: 0.285313, dice: 0.301506, loss: 0.293410
train iou: 0.7575675648752941 dice: 0.8106936657800263
valid iou: 0.7756246023866029 dice: 0.8350271983174506
10m 60s
Epoch 3/99
----------
LR 0.0001
train: bce: 0.299572, dice: 0.313092, loss: 0.306332
val: bce: 0.317161, dice: 0.307548, loss: 0.312354
train iou: 0.7643872462664943 dice: 0.8171284784422946
valid

train: bce: 0.096965, dice: 0.072752, loss: 0.084859
val: bce: 0.343400, dice: 0.143221, loss: 0.243310
train iou: 0.9151035587317056 dice: 0.94110243309831
valid iou: 0.8337975856616363 dice: 0.8901132115149701
10m 44s
Epoch 33/99
----------
LR 1e-05
train: bce: 0.095624, dice: 0.072796, loss: 0.084210
val: bce: 0.333145, dice: 0.142262, loss: 0.237704
train iou: 0.9162139797486951 dice: 0.9418113339973803
valid iou: 0.8341218471787265 dice: 0.8912615093420696
10m 43s
Epoch 34/99
----------
LR 1e-05
train: bce: 0.092180, dice: 0.069895, loss: 0.081037
val: bce: 0.392569, dice: 0.149228, loss: 0.270898
train iou: 0.9205731924789543 dice: 0.9456315294344081
valid iou: 0.8233894392608498 dice: 0.8844577804053144
10m 39s
Epoch 35/99
----------
LR 1e-05
train: bce: 0.090702, dice: 0.066017, loss: 0.078359
val: bce: 0.376731, dice: 0.140328, loss: 0.258530
train iou: 0.9206694296656186 dice: 0.9456787547410656
valid iou: 0.8293517319052154 dice: 0.8894420380558
10m 42s
Epoch 36/99
---------

train: bce: 0.070917, dice: 0.048957, loss: 0.059937
val: bce: 0.385973, dice: 0.139717, loss: 0.262845
train iou: 0.9364199740786185 dice: 0.9566311720127628
valid iou: 0.8379134631729513 dice: 0.8999563755947456
10m 39s
Epoch 65/99
----------
LR 1.0000000000000002e-06
train: bce: 0.067456, dice: 0.047424, loss: 0.057440
val: bce: 0.416057, dice: 0.139269, loss: 0.277663
train iou: 0.9389906439381842 dice: 0.9582859539312418
valid iou: 0.8310098647497377 dice: 0.8943954016093424
10m 38s
Epoch 66/99
----------
LR 1.0000000000000002e-06
train: bce: 0.065360, dice: 0.047569, loss: 0.056464
val: bce: 0.422666, dice: 0.143638, loss: 0.283152
train iou: 0.940355085309995 dice: 0.9599198028388556
valid iou: 0.8311496839939594 dice: 0.8951793135464153
10m 43s
Epoch 67/99
----------
LR 1.0000000000000002e-06
train: bce: 0.069357, dice: 0.047694, loss: 0.058526
val: bce: 0.384407, dice: 0.138253, loss: 0.261330
train iou: 0.9383470095066685 dice: 0.9581378050564363
valid iou: 0.8322267331305521

train: bce: 0.063959, dice: 0.044720, loss: 0.054340
val: bce: 0.384028, dice: 0.137630, loss: 0.260829
train iou: 0.9422312063301882 dice: 0.9609723359216578
valid iou: 0.8388779598570938 dice: 0.8977581655969299
11m 10s
Epoch 96/99
----------
LR 1.0000000000000002e-07
train: bce: 0.066814, dice: 0.045170, loss: 0.055992
val: bce: 0.404558, dice: 0.137397, loss: 0.270977
train iou: 0.9409616119477716 dice: 0.9605253934476788
valid iou: 0.8362435380518933 dice: 0.8982852161999859
11m 11s
Epoch 97/99
----------
LR 1.0000000000000002e-07
train: bce: 0.065121, dice: 0.045672, loss: 0.055396
val: bce: 0.410670, dice: 0.140940, loss: 0.275805
train iou: 0.9405433168699016 dice: 0.9599049696650304
valid iou: 0.8343901569876779 dice: 0.8978245737186141
11m 5s
Epoch 98/99
----------
LR 1.0000000000000002e-07
train: bce: 0.065850, dice: 0.045318, loss: 0.055584
val: bce: 0.358806, dice: 0.141014, loss: 0.249910
train iou: 0.940439322357385 dice: 0.9594077895794474
valid iou: 0.8353131733562451 

In [89]:
torch.cuda.empty_cache()

test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)

test: bce: 0.449919, dice: 0.140567, loss: 0.295243
test_model ## iou: 0.8246404832795897 dice: 0.8921236089646206
test: bce: 0.449919, dice: 0.140567, loss: 0.295243
test_model ## iou: 0.822614630366076 dice: 0.889411455758269
test: bce: 0.449919, dice: 0.140567, loss: 0.295243
test_model ## iou: 0.8220569760130546 dice: 0.8900706764873945
test: bce: 0.449919, dice: 0.140567, loss: 0.295243
test_model ## iou: 0.8226969660833403 dice: 0.8901469804407551
test: bce: 0.449919, dice: 0.140567, loss: 0.295243
test_model ## iou: 0.825096038882489 dice: 0.8927429289831981


최종 model train 

In [9]:
image_datasets = {
    'train': trainDataset, 'val': validDataset
}

dataloaders = {
    'train': trainLoader,
    'val': validLoader
}

dataset_sizes = {
    x: len(image_datasets[x]) for x in image_datasets.keys()
}

dataset_sizes


from collections import defaultdict
import torch.nn.functional as F
# from loss import dice_loss


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def iou_score(output, target):
    smooth = 1e-5

    if torch.is_tensor(output):
        output = torch.sigmoid(output).data.cpu().numpy()
    if torch.is_tensor(target):
        target = target.data.cpu().numpy()
    output_ = output > 0.5
    target_ = target > 0.5
    intersection = (output_ & target_).sum()
    union = (output_ | target_).sum()

    return (intersection + smooth) / (union + smooth)


def dice_coef(output, target):
    smooth = 1e-5

    output = torch.sigmoid(output).view(-1).data.cpu().numpy()
    target = target.view(-1).data.cpu().numpy()
    intersection = (output * target).sum()

    return (2. * intersection + smooth) / \
        (output.sum() + target.sum() + smooth)


def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()    

    intersection = (pred * target).sum(dim=2).sum(dim=2)
    
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    
    return loss.mean()


def calc_loss(pred, target, metrics, bce_weight=0.5):   
    bce = F.binary_cross_entropy_with_logits(pred, target)
        
    pred = F.sigmoid(pred)
    dice = dice_loss(pred, target)
    
    loss = bce * bce_weight + dice * (1 - bce_weight)
    
    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)

    
    return loss


def print_metrics(metrics, epoch_samples, phase = 'test',history=None):    
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
        
    print("{}: {}".format(phase, ", ".join(outputs)))    
    if history:
        if phase == 'train':
            history['loss'].append(metrics['loss']/epoch_samples)
            history['bce'].append(metrics['bce']/epoch_samples)
            history['dice'].append(metrics['dice']/epoch_samples)
        
        elif phase == 'val':
            history['val_loss'].append(metrics['loss']/epoch_samples)
            history['val_bce'].append(metrics['bce']/epoch_samples)
            history['val_dice'].append(metrics['dice']/epoch_samples)


def test_model(model,loader):
    model.eval()
    
    metrics = defaultdict(float)
    epoch_samples = 0
    iouMeter = AverageMeter()
    diceMeter = AverageMeter()
    with torch.no_grad():
        for rgb,thr,labels,danger in loader:
            rgb = rgb.to(device)
            thr = thr.to(device)
            labels = labels.to(device)             

            outputs = model(rgb,thr)

            loss = calc_loss(outputs, labels, metrics)
            epoch_samples += rgb.size(0)

            iou = iou_score(outputs,labels)
            dice = dice_coef(outputs,labels)
            iouMeter.update(iou, rgb.size(0))
            diceMeter.update(dice, rgb.size(0))

        print_metrics(metrics, epoch_samples,'test')
        print('test_model ##','iou:',iouMeter.avg,'dice:',diceMeter.avg)
        epoch_loss = metrics['loss'] / epoch_samples

    
    
def train_model(model, optimizer, scheduler, num_epochs=25,early_stop=False):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    
    history = {'loss':[],'bce':[],'dice':[],'iou':[],'dice_coef':[],
              'val_loss':[],'val_bce':[],'val_dice':[],'val_iou':[],'val_dice_coef':[]}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        since = time.time()
        
        iouMeter = AverageMeter()
        diceMeter = AverageMeter()
        valid_iouMeter = AverageMeter()
        valid_diceMeter = AverageMeter()
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])
                    
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0
            
            for rgb,thr, labels,danger in dataloaders[phase]:
                
                rgb = rgb.to(device)
                thr = thr.to(device)
                labels = labels.to(device)             

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(rgb,thr)
                    loss = calc_loss(outputs, labels, metrics)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += rgb.size(0)
                
                if phase == 'train':
                    iou = iou_score(outputs,labels)
                    dice = dice_coef(outputs,labels)
                    iouMeter.update(iou, rgb.size(0))
                    diceMeter.update(dice, rgb.size(0))
                    
                if phase == 'val':
                    iou = iou_score(outputs,labels)
                    dice = dice_coef(outputs,labels)
                    valid_iouMeter.update(iou, rgb.size(0))
                    valid_diceMeter.update(dice, rgb.size(0))

            print_metrics(metrics, epoch_samples, phase,history)
            epoch_loss = metrics['loss'] / epoch_samples
            
            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print('train iou:',iouMeter.avg,'dice:',diceMeter.avg)
        print('valid iou:',valid_iouMeter.avg,'dice:',valid_diceMeter.avg)
        history['iou'].append(iouMeter.avg)
        history['dice_coef'].append(diceMeter.avg)
        history['val_iou'].append(valid_iouMeter.avg)
        history['val_dice_coef'].append(valid_diceMeter.avg)
            
        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    if early_stop:
        model.load_state_dict(best_model_wts)
    return model,history

마지막에만 fuse 하는 모델 

In [13]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
import math



class Swish(nn.Module):

    def __init__(self, *args, **kwargs):
        super(Swish, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)


class ConvBNReLU(nn.Sequential):

    def __init__(self, in_planes, out_planes, kernel_size, stride=1, groups=1):
        padding = self._get_padding(kernel_size, stride)
        super(ConvBNReLU, self).__init__(
            nn.ZeroPad2d(padding),
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding=0, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes),
            Swish(),
        )

    def _get_padding(self, kernel_size, stride):
        p = max(kernel_size - stride, 0)
        return [p // 2, p - p // 2, p // 2, p - p // 2]

class SqueezeExcitation(nn.Module):

    def __init__(self, in_planes, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_planes, reduced_dim, 1),
            Swish(),
            nn.Conv2d(reduced_dim, in_planes, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)

class MBConvBlock(nn.Module):

    def __init__(self,
                 in_planes,
                 out_planes,
                 expand_ratio,
                 kernel_size,
                 stride,
                 reduction_ratio=4,
                 drop_connect_rate=0.2):
        super(MBConvBlock, self).__init__()
        self.drop_connect_rate = drop_connect_rate
        self.use_residual = in_planes == out_planes and stride == 1
        assert stride in [1, 2]
        assert kernel_size in [3, 5]

        hidden_dim = in_planes * expand_ratio
        reduced_dim = max(1, int(in_planes / reduction_ratio))

        layers = []
        # pw
        if in_planes != hidden_dim:
            layers += [ConvBNReLU(in_planes, hidden_dim, 1)]

        layers += [
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, kernel_size, stride=stride, groups=hidden_dim),
            # se
            SqueezeExcitation(hidden_dim, reduced_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, out_planes, 1, bias=False),
            nn.BatchNorm2d(out_planes),
        ]

        self.conv = nn.Sequential(*layers)

    def _drop_connect(self, x):
        if not self.training:
            return x
        keep_prob = 1.0 - self.drop_connect_rate
        batch_size = x.size(0)
        random_tensor = keep_prob
        random_tensor += torch.rand(batch_size, 1, 1, 1, device=x.device)
        binary_tensor = random_tensor.floor()
        return x.div(keep_prob) * binary_tensor

    def forward(self, x):
        if self.use_residual:
            return x + self._drop_connect(self.conv(x))
        else:
            return self.conv(x)
        


class ResConvBlock(torch.nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.layer = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
            torch.nn.Conv2d(in_channels=out_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
        ])
        self.shortcut = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                           out_channels=out_channels,
                           kernel_size=(1,1),stride=1),
            torch.nn.BatchNorm2d(out_channels)
        ])
    def forward(self,x):
        shortcut = self.shortcut(x)
        x = self.layer(x)
        return x + shortcut

    
class FusionBlock(torch.nn.Module):
    def __init__(self,channel,height,width):
        super(FusionBlock, self).__init__()
        self.weightT = torch.nn.Parameter(torch.rand(size=(channel,height,width),requires_grad = True))
        self.weightR = torch.nn.Parameter(torch.rand(size=(channel,height,width),requires_grad = True))
        
    def forward(self,thr,rgb):
        thrAlpha = torch.sigmoid(self.weightT)
        thrBeta = 1-thrAlpha
        
        rgbAlpha = torch.sigmoid(self.weightR)
        rgbBeta = 1-rgbAlpha
        
        thr_ = thrAlpha*thr + thrBeta*rgb
        rgb_ = rgbAlpha*rgb + rgbBeta*thr
        
        return thr_,rgb_

class MBConvIntertwinedUNet(torch.nn.Module):
    def __init__(self, n_class):
        super().__init__()
        self.n_features = [64,256,256,256,256]
        self.downsample = torch.nn.MaxPool2d(2)
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        

        self.rgbStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        self.thrStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        

        self.rgbLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])
        
        self.thrLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])
        self.thrLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        
        self.rgbLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])

        self.bridge = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[4]*2,out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1)

            ])
        
        self.fuse1 = FusionBlock(channel=self.n_features[0],height=128//2,width=256//2)
        self.fuse2 = FusionBlock(channel=self.n_features[1],height=128//4,width=256//4)
        self.fuse3 = FusionBlock(channel=self.n_features[2],height=128//8,width=256//8)
        self.fuse4 = FusionBlock(channel=self.n_features[3],height=128//16,width=256//16)
        
        
        self.dLayer4 = ResConvBlock(self.n_features[4]+self.n_features[3]*2,self.n_features[3])
        self.dLayer3 = ResConvBlock(self.n_features[3]+self.n_features[2]*2,self.n_features[2])
        self.dLayer2 = ResConvBlock(self.n_features[2]+self.n_features[1]*2,self.n_features[1])
        self.dLayer1 = ResConvBlock(self.n_features[1]+self.n_features[0]*2,self.n_features[0])
        
        self.out = torch.nn.Conv2d(self.n_features[0],n_class,1)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                fan_out = m.weight.size(0)
                init_range = 1.0 / math.sqrt(fan_out)
                nn.init.uniform_(m.weight, -init_range, init_range)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        
    def forward(self, rgb, thr):
        thrStem = self.thrStem(thr)
        rgbStem = self.rgbStem(rgb)  # 256 
        
        
        thr = self.downsample(thrStem)
        rgb = self.downsample(rgbStem) # 128
        #thr,rgb = self.fuse1(thr,rgb)
        
        thrLayer1 = self.thrLayer1(thr)
        rgbLayer1 = self.rgbLayer1(rgb)
        
        
        thr = self.downsample(thrLayer1)
        rgb = self.downsample(rgbLayer1) # 64 
        #thr,rgb = self.fuse2(thr,rgb)
        
        thrLayer2 = self.thrLayer2(thr)
        rgbLayer2 = self.rgbLayer2(rgb)
        
        
        thr = self.downsample(thrLayer2)
        rgb = self.downsample(rgbLayer2) # 32 
        #thr,rgb = self.fuse3(thr,rgb)
        
        thrLayer3 = self.thrLayer3(thr)
        rgbLayer3 = self.rgbLayer3(rgb)
        
        
        thr = self.downsample(thrLayer3)
        rgb = self.downsample(rgbLayer3) # 16
        thr,rgb = self.fuse4(thr,rgb)
        
        thrLayer4 = self.thrLayer4(thr)
        rgbLayer4 = self.rgbLayer4(rgb)

        sumLayer4 = self.bridge(torch.cat([rgbLayer4,thrLayer4],dim=1))
        
        
        x = self.upsample(sumLayer4)
        x = torch.cat([x,rgbLayer3,thrLayer3],dim=1)
        x = self.dLayer4(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer2,thrLayer2],dim=1)
        x = self.dLayer3(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer1,thrLayer1],dim=1)
        x = self.dLayer2(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbStem,thrStem],dim=1)
        x = self.dLayer1(x)
        
        out = self.out(x)
        
        return out
        
model = MBConvIntertwinedUNet(n_class=1)



In [14]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_class = 1

model = MBConvIntertwinedUNet(n_class=1).to(device)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=25, gamma=0.1)

model,history = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=100)

cuda:0
Epoch 0/99
----------
LR 0.0001


/compuworks/anaconda3/envs/py36torch/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/compuworks/anaconda3/envs/py36torch/lib/python3.6/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train: bce: 0.490955, dice: 0.351764, loss: 0.421360
val: bce: 0.295519, dice: 0.346299, loss: 0.320909
saving best model
train iou: 0.6802637431136762 dice: 0.7579810001783349
valid iou: 0.7520200728900064 dice: 0.7797368198788892
9m 54s
Epoch 1/99
----------
LR 0.0001
train: bce: 0.352077, dice: 0.327449, loss: 0.339763
val: bce: 0.287060, dice: 0.312994, loss: 0.300027
saving best model
train iou: 0.7333891158968909 dice: 0.7895881132800141
valid iou: 0.7698165822211009 dice: 0.8152209711787425
10m 5s
Epoch 2/99
----------
LR 0.0001
train: bce: 0.318349, dice: 0.316993, loss: 0.317671
val: bce: 0.275005, dice: 0.308986, loss: 0.291996
saving best model
train iou: 0.7560098956863964 dice: 0.8088331733596685
valid iou: 0.7742983031719457 dice: 0.8263824638952749
10m 9s
Epoch 3/99
----------
LR 0.0001
train: bce: 0.299973, dice: 0.311612, loss: 0.305793
val: bce: 0.262959, dice: 0.299506, loss: 0.281233
saving best model
train iou: 0.7647709167318276 dice: 0.8147542820479363
valid iou:

train: bce: 0.094374, dice: 0.072183, loss: 0.083279
val: bce: 0.387989, dice: 0.140608, loss: 0.264299
train iou: 0.9183569031729478 dice: 0.943716621557492
valid iou: 0.832001166816528 dice: 0.8936012070985454
9m 40s
Epoch 33/99
----------
LR 1e-05
train: bce: 0.093824, dice: 0.073701, loss: 0.083763
val: bce: 0.415858, dice: 0.142775, loss: 0.279317
train iou: 0.9174131290278715 dice: 0.9417295937916693
valid iou: 0.8288442704595197 dice: 0.8904267068805912
9m 38s
Epoch 34/99
----------
LR 1e-05
train: bce: 0.096001, dice: 0.073605, loss: 0.084803
val: bce: 0.327449, dice: 0.147711, loss: 0.237580
train iou: 0.9184660854929552 dice: 0.9435705899825313
valid iou: 0.8362121556377059 dice: 0.8936368273098365
9m 36s
Epoch 35/99
----------
LR 1e-05
train: bce: 0.089661, dice: 0.066889, loss: 0.078275
val: bce: 0.383208, dice: 0.144861, loss: 0.264035
train iou: 0.9225145134742931 dice: 0.9470703901295583
valid iou: 0.8326985908610998 dice: 0.8935070943637919
9m 36s
Epoch 36/99
----------

train: bce: 0.066545, dice: 0.047419, loss: 0.056982
val: bce: 0.391685, dice: 0.144685, loss: 0.268185
train iou: 0.9399433115265439 dice: 0.9594658901436252
valid iou: 0.8339458524421909 dice: 0.8979253714148449
9m 55s
Epoch 66/99
----------
LR 1.0000000000000002e-06
train: bce: 0.065676, dice: 0.047316, loss: 0.056496
val: bce: 0.400187, dice: 0.145845, loss: 0.273016
train iou: 0.9397693294685553 dice: 0.9587374629798949
valid iou: 0.8342396530397118 dice: 0.8973608123925879
9m 54s
Epoch 67/99
----------
LR 1.0000000000000002e-06
train: bce: 0.066263, dice: 0.048806, loss: 0.057535
val: bce: 0.400588, dice: 0.143460, loss: 0.272024
train iou: 0.9395473533643312 dice: 0.9587760363452682
valid iou: 0.8383075015603069 dice: 0.9007794350618495
9m 55s
Epoch 68/99
----------
LR 1.0000000000000002e-06
train: bce: 0.065196, dice: 0.046130, loss: 0.055663
val: bce: 0.361390, dice: 0.141816, loss: 0.251603
train iou: 0.9409277608572937 dice: 0.9599012032287748
valid iou: 0.8355780643041146 d

train: bce: 0.063391, dice: 0.044579, loss: 0.053985
val: bce: 0.425117, dice: 0.145684, loss: 0.285400
train iou: 0.9427141311655757 dice: 0.9612424261204894
valid iou: 0.8278057994066798 dice: 0.8915828783482449
9m 55s
Epoch 97/99
----------
LR 1.0000000000000002e-07
train: bce: 0.065729, dice: 0.047265, loss: 0.056497
val: bce: 0.454059, dice: 0.147556, loss: 0.300807
train iou: 0.9408469732138655 dice: 0.959741504400752
valid iou: 0.8240347433480825 dice: 0.8894520971422424
9m 56s
Epoch 98/99
----------
LR 1.0000000000000002e-07
train: bce: 0.064645, dice: 0.045649, loss: 0.055147
val: bce: 0.366603, dice: 0.146787, loss: 0.256695
train iou: 0.9408082629499672 dice: 0.9597834421897556
valid iou: 0.8350705169992446 dice: 0.8965542452646473
9m 56s
Epoch 99/99
----------
LR 1.0000000000000004e-08
train: bce: 0.063222, dice: 0.046073, loss: 0.054648
val: bce: 0.429417, dice: 0.143175, loss: 0.286296
train iou: 0.9425193432659068 dice: 0.9613451838300148
valid iou: 0.8352935433642971 di

In [15]:
torch.cuda.empty_cache()

test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)

test: bce: 0.435860, dice: 0.142679, loss: 0.289269
test_model ## iou: 0.833675553279857 dice: 0.897619094496167
test: bce: 0.435860, dice: 0.142679, loss: 0.289269
test_model ## iou: 0.8308262420831064 dice: 0.8955522144563954
test: bce: 0.435860, dice: 0.142679, loss: 0.289269
test_model ## iou: 0.8333416421249485 dice: 0.897285983209593
test: bce: 0.435860, dice: 0.142679, loss: 0.289269
test_model ## iou: 0.8328410023725071 dice: 0.8967103248620164
test: bce: 0.435860, dice: 0.142679, loss: 0.289269
test_model ## iou: 0.8312192726911194 dice: 0.8959378140690457


In [16]:
MBConvNet = model.cpu()

In [17]:
torch.save(MBConvNet.state_dict(),'./MBConvNetFuse4.pt')

In [18]:
import os
os.mkdir('./model_history_files/')

In [22]:
import pickle
with open('./model_history_files/MBConvNetFuse4.bin','wb') as f:
    pickle.dump(history,f)

3 4 층에서 fuse 

In [15]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
import math



class Swish(nn.Module):

    def __init__(self, *args, **kwargs):
        super(Swish, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)


class ConvBNReLU(nn.Sequential):

    def __init__(self, in_planes, out_planes, kernel_size, stride=1, groups=1):
        padding = self._get_padding(kernel_size, stride)
        super(ConvBNReLU, self).__init__(
            nn.ZeroPad2d(padding),
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding=0, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes),
            Swish(),
        )

    def _get_padding(self, kernel_size, stride):
        p = max(kernel_size - stride, 0)
        return [p // 2, p - p // 2, p // 2, p - p // 2]

class SqueezeExcitation(nn.Module):

    def __init__(self, in_planes, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_planes, reduced_dim, 1),
            Swish(),
            nn.Conv2d(reduced_dim, in_planes, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)

class MBConvBlock(nn.Module):

    def __init__(self,
                 in_planes,
                 out_planes,
                 expand_ratio,
                 kernel_size,
                 stride,
                 reduction_ratio=4,
                 drop_connect_rate=0.2):
        super(MBConvBlock, self).__init__()
        self.drop_connect_rate = drop_connect_rate
        self.use_residual = in_planes == out_planes and stride == 1
        assert stride in [1, 2]
        assert kernel_size in [3, 5]

        hidden_dim = in_planes * expand_ratio
        reduced_dim = max(1, int(in_planes / reduction_ratio))

        layers = []
        # pw
        if in_planes != hidden_dim:
            layers += [ConvBNReLU(in_planes, hidden_dim, 1)]

        layers += [
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, kernel_size, stride=stride, groups=hidden_dim),
            # se
            SqueezeExcitation(hidden_dim, reduced_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, out_planes, 1, bias=False),
            nn.BatchNorm2d(out_planes),
        ]

        self.conv = nn.Sequential(*layers)

    def _drop_connect(self, x):
        if not self.training:
            return x
        keep_prob = 1.0 - self.drop_connect_rate
        batch_size = x.size(0)
        random_tensor = keep_prob
        random_tensor += torch.rand(batch_size, 1, 1, 1, device=x.device)
        binary_tensor = random_tensor.floor()
        return x.div(keep_prob) * binary_tensor

    def forward(self, x):
        if self.use_residual:
            return x + self._drop_connect(self.conv(x))
        else:
            return self.conv(x)
        


class ResConvBlock(torch.nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.layer = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
            torch.nn.Conv2d(in_channels=out_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
        ])
        self.shortcut = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                           out_channels=out_channels,
                           kernel_size=(1,1),stride=1),
            torch.nn.BatchNorm2d(out_channels)
        ])
    def forward(self,x):
        shortcut = self.shortcut(x)
        x = self.layer(x)
        return x + shortcut

    
class FusionBlock(torch.nn.Module):
    def __init__(self,channel,height,width):
        super(FusionBlock, self).__init__()
        self.weightT = torch.nn.Parameter(torch.rand(size=(channel,height,width),requires_grad = True))
        self.weightR = torch.nn.Parameter(torch.rand(size=(channel,height,width),requires_grad = True))
        
    def forward(self,thr,rgb):
        thrAlpha = torch.sigmoid(self.weightT)
        thrBeta = 1-thrAlpha
        
        rgbAlpha = torch.sigmoid(self.weightR)
        rgbBeta = 1-rgbAlpha
        
        thr_ = thrAlpha*thr + thrBeta*rgb
        rgb_ = rgbAlpha*rgb + rgbBeta*thr
        
        return thr_,rgb_

class MBConvIntertwinedUNet(torch.nn.Module):
    def __init__(self, n_class):
        super().__init__()
        self.n_features = [64,256,256,256,256]
        self.downsample = torch.nn.MaxPool2d(2)
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        

        self.rgbStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        self.thrStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        

        self.rgbLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])
        
        self.thrLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])
        self.thrLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        
        self.rgbLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])

        self.bridge = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[4]*2,out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1)

            ])
        
        self.fuse1 = FusionBlock(channel=self.n_features[0],height=128//2,width=256//2)
        self.fuse2 = FusionBlock(channel=self.n_features[1],height=128//4,width=256//4)
        self.fuse3 = FusionBlock(channel=self.n_features[2],height=128//8,width=256//8)
        self.fuse4 = FusionBlock(channel=self.n_features[3],height=128//16,width=256//16)
        
        
        self.dLayer4 = ResConvBlock(self.n_features[4]+self.n_features[3]*2,self.n_features[3])
        self.dLayer3 = ResConvBlock(self.n_features[3]+self.n_features[2]*2,self.n_features[2])
        self.dLayer2 = ResConvBlock(self.n_features[2]+self.n_features[1]*2,self.n_features[1])
        self.dLayer1 = ResConvBlock(self.n_features[1]+self.n_features[0]*2,self.n_features[0])
        
        self.out = torch.nn.Conv2d(self.n_features[0],n_class,1)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                fan_out = m.weight.size(0)
                init_range = 1.0 / math.sqrt(fan_out)
                nn.init.uniform_(m.weight, -init_range, init_range)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        
    def forward(self, rgb, thr):
        thrStem = self.thrStem(thr)
        rgbStem = self.rgbStem(rgb)  # 256 
        
        
        thr = self.downsample(thrStem)
        rgb = self.downsample(rgbStem) # 128
        #thr,rgb = self.fuse1(thr,rgb)
        
        thrLayer1 = self.thrLayer1(thr)
        rgbLayer1 = self.rgbLayer1(rgb)
        
        
        thr = self.downsample(thrLayer1)
        rgb = self.downsample(rgbLayer1) # 64 
#         thr,rgb = self.fuse2(thr,rgb)
        
        thrLayer2 = self.thrLayer2(thr)
        rgbLayer2 = self.rgbLayer2(rgb)
        
        
        thr = self.downsample(thrLayer2)
        rgb = self.downsample(rgbLayer2) # 32 
        thr,rgb = self.fuse3(thr,rgb)
        
        thrLayer3 = self.thrLayer3(thr)
        rgbLayer3 = self.rgbLayer3(rgb)
        
        
        thr = self.downsample(thrLayer3)
        rgb = self.downsample(rgbLayer3) # 16
        thr,rgb = self.fuse4(thr,rgb)
        
        thrLayer4 = self.thrLayer4(thr)
        rgbLayer4 = self.rgbLayer4(rgb)

        sumLayer4 = self.bridge(torch.cat([rgbLayer4,thrLayer4],dim=1))
        
        
        x = self.upsample(sumLayer4)
        x = torch.cat([x,rgbLayer3,thrLayer3],dim=1)
        x = self.dLayer4(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer2,thrLayer2],dim=1)
        x = self.dLayer3(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer1,thrLayer1],dim=1)
        x = self.dLayer2(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbStem,thrStem],dim=1)
        x = self.dLayer1(x)
        
        out = self.out(x)
        
        return out
        
model = MBConvIntertwinedUNet(n_class=1)



3 4 fuse, stpe size 50, gamma 0.5

In [16]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_class = 1

model = MBConvIntertwinedUNet(n_class=1).to(device)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=50, gamma=0.5)

model,history = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=100)

cuda:0
Epoch 0/99
----------
LR 0.0001
train: bce: 0.526162, dice: 0.354221, loss: 0.440191
val: bce: 0.372062, dice: 0.313495, loss: 0.342778
saving best model
train iou: 0.6759374438247585 dice: 0.7581090317932117
valid iou: 0.7472452653428424 dice: 0.8153670703996281
9m 3s
Epoch 1/99
----------
LR 0.0001
train: bce: 0.344886, dice: 0.325630, loss: 0.335258
val: bce: 0.286458, dice: 0.306160, loss: 0.296309
saving best model
train iou: 0.7348280111186578 dice: 0.795002076660585
valid iou: 0.7726423262022231 dice: 0.8293010068988285
9m 10s
Epoch 2/99
----------
LR 0.0001
train: bce: 0.320868, dice: 0.317737, loss: 0.319302
val: bce: 0.283694, dice: 0.302679, loss: 0.293186
saving best model
train iou: 0.749214128476288 dice: 0.806995354774599
valid iou: 0.7848261647897055 dice: 0.8424548696014772
9m 10s
Epoch 3/99
----------
LR 0.0001
train: bce: 0.301333, dice: 0.311611, loss: 0.306472
val: bce: 0.270089, dice: 0.315939, loss: 0.293014
saving best model
train iou: 0.7619569738275764 

train: bce: 0.135190, dice: 0.108562, loss: 0.121876
val: bce: 0.696498, dice: 0.161875, loss: 0.429186
train iou: 0.8894651973448778 dice: 0.9219277298724022
valid iou: 0.798636054036373 dice: 0.864874157267294
9m 14s
Epoch 33/99
----------
LR 0.0001
train: bce: 0.130320, dice: 0.101514, loss: 0.115917
val: bce: 0.327748, dice: 0.165265, loss: 0.246507
train iou: 0.8924948644432193 dice: 0.9250409550556679
valid iou: 0.8310966997940161 dice: 0.8873401263659122
9m 14s
Epoch 34/99
----------
LR 0.0001
train: bce: 0.133722, dice: 0.099956, loss: 0.116839
val: bce: 0.409668, dice: 0.144351, loss: 0.277009
train iou: 0.8934940435366808 dice: 0.9248351597689028
valid iou: 0.8298901194225132 dice: 0.8870159030513576
9m 14s
Epoch 35/99
----------
LR 0.0001
train: bce: 0.124289, dice: 0.092444, loss: 0.108366
val: bce: 0.454801, dice: 0.150389, loss: 0.302595
train iou: 0.8985159792084578 dice: 0.9289725027472169
valid iou: 0.8229002118481613 dice: 0.8841046771418458
9m 14s
Epoch 36/99
-------

train: bce: 0.049583, dice: 0.036916, loss: 0.043250
val: bce: 0.426958, dice: 0.129635, loss: 0.278296
train iou: 0.9539588021988019 dice: 0.9689514726101417
valid iou: 0.8426141005347917 dice: 0.9050191541389205
9m 15s
Epoch 66/99
----------
LR 5e-05
train: bce: 0.053557, dice: 0.038948, loss: 0.046252
val: bce: 0.464861, dice: 0.130778, loss: 0.297820
train iou: 0.9518910046722427 dice: 0.9673945830086937
valid iou: 0.8390219491690166 dice: 0.9036608511451472
9m 16s
Epoch 67/99
----------
LR 5e-05
train: bce: 0.048239, dice: 0.034448, loss: 0.041343
val: bce: 0.459354, dice: 0.124361, loss: 0.291858
train iou: 0.9557701394847887 dice: 0.970907423058762
valid iou: 0.846612575249157 dice: 0.9078149745417338
9m 16s
Epoch 68/99
----------
LR 5e-05
train: bce: 0.046310, dice: 0.032596, loss: 0.039453
val: bce: 0.504560, dice: 0.129033, loss: 0.316796
train iou: 0.9573719577771593 dice: 0.9717996513018261
valid iou: 0.8361928118015962 dice: 0.9013038100799168
9m 16s
Epoch 69/99
----------

train: bce: 0.033754, dice: 0.025811, loss: 0.029783
val: bce: 0.555791, dice: 0.131878, loss: 0.343835
train iou: 0.966999362782506 dice: 0.9778253547564738
valid iou: 0.8335563935000019 dice: 0.8997427139304852
9m 19s
Epoch 99/99
----------
LR 2.5e-05
train: bce: 0.029932, dice: 0.023398, loss: 0.026665
val: bce: 0.532474, dice: 0.128836, loss: 0.330655
train iou: 0.9712712094550294 dice: 0.9809478996475671
valid iou: 0.8367832730070122 dice: 0.9018434325268511
9m 19s
Best val loss: 0.228381


In [17]:
torch.cuda.empty_cache()

test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)

test: bce: 0.590869, dice: 0.124264, loss: 0.357566
test_model ## iou: 0.828727509198773 dice: 0.8964463807774534
test: bce: 0.590869, dice: 0.124264, loss: 0.357566
test_model ## iou: 0.830293665301716 dice: 0.8981994800650712
test: bce: 0.590869, dice: 0.124264, loss: 0.357566
test_model ## iou: 0.8290414023256578 dice: 0.8949736669124816
test: bce: 0.590869, dice: 0.124264, loss: 0.357566
test_model ## iou: 0.8293634691155181 dice: 0.8975422012988983
test: bce: 0.590869, dice: 0.124264, loss: 0.357566
test_model ## iou: 0.829571509260186 dice: 0.8971667629410756


In [18]:
model = model.cpu()

torch.save(model.state_dict(),'./MBConvNetFuse34_gamma05.pt')

import pickle
with open('./model_history_files/MBConvNetFuse34_gamma05.bin','wb') as f:
    pickle.dump(history,f)

1 4 fuse, stpe size 50, gamma 0.5

In [19]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
import math



class Swish(nn.Module):

    def __init__(self, *args, **kwargs):
        super(Swish, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)


class ConvBNReLU(nn.Sequential):

    def __init__(self, in_planes, out_planes, kernel_size, stride=1, groups=1):
        padding = self._get_padding(kernel_size, stride)
        super(ConvBNReLU, self).__init__(
            nn.ZeroPad2d(padding),
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding=0, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes),
            Swish(),
        )

    def _get_padding(self, kernel_size, stride):
        p = max(kernel_size - stride, 0)
        return [p // 2, p - p // 2, p // 2, p - p // 2]

class SqueezeExcitation(nn.Module):

    def __init__(self, in_planes, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_planes, reduced_dim, 1),
            Swish(),
            nn.Conv2d(reduced_dim, in_planes, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)

class MBConvBlock(nn.Module):

    def __init__(self,
                 in_planes,
                 out_planes,
                 expand_ratio,
                 kernel_size,
                 stride,
                 reduction_ratio=4,
                 drop_connect_rate=0.2):
        super(MBConvBlock, self).__init__()
        self.drop_connect_rate = drop_connect_rate
        self.use_residual = in_planes == out_planes and stride == 1
        assert stride in [1, 2]
        assert kernel_size in [3, 5]

        hidden_dim = in_planes * expand_ratio
        reduced_dim = max(1, int(in_planes / reduction_ratio))

        layers = []
        # pw
        if in_planes != hidden_dim:
            layers += [ConvBNReLU(in_planes, hidden_dim, 1)]

        layers += [
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, kernel_size, stride=stride, groups=hidden_dim),
            # se
            SqueezeExcitation(hidden_dim, reduced_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, out_planes, 1, bias=False),
            nn.BatchNorm2d(out_planes),
        ]

        self.conv = nn.Sequential(*layers)

    def _drop_connect(self, x):
        if not self.training:
            return x
        keep_prob = 1.0 - self.drop_connect_rate
        batch_size = x.size(0)
        random_tensor = keep_prob
        random_tensor += torch.rand(batch_size, 1, 1, 1, device=x.device)
        binary_tensor = random_tensor.floor()
        return x.div(keep_prob) * binary_tensor

    def forward(self, x):
        if self.use_residual:
            return x + self._drop_connect(self.conv(x))
        else:
            return self.conv(x)
        


class ResConvBlock(torch.nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.layer = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
            torch.nn.Conv2d(in_channels=out_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
        ])
        self.shortcut = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                           out_channels=out_channels,
                           kernel_size=(1,1),stride=1),
            torch.nn.BatchNorm2d(out_channels)
        ])
    def forward(self,x):
        shortcut = self.shortcut(x)
        x = self.layer(x)
        return x + shortcut

    
class FusionBlock(torch.nn.Module):
    def __init__(self,channel,height,width):
        super(FusionBlock, self).__init__()
        self.weightT = torch.nn.Parameter(torch.rand(size=(channel,height,width),requires_grad = True))
        self.weightR = torch.nn.Parameter(torch.rand(size=(channel,height,width),requires_grad = True))
        
    def forward(self,thr,rgb):
        thrAlpha = torch.sigmoid(self.weightT)
        thrBeta = 1-thrAlpha
        
        rgbAlpha = torch.sigmoid(self.weightR)
        rgbBeta = 1-rgbAlpha
        
        thr_ = thrAlpha*thr + thrBeta*rgb
        rgb_ = rgbAlpha*rgb + rgbBeta*thr
        
        return thr_,rgb_

class MBConvIntertwinedUNet(torch.nn.Module):
    def __init__(self, n_class):
        super().__init__()
        self.n_features = [64,256,256,256,256]
        self.downsample = torch.nn.MaxPool2d(2)
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        

        self.rgbStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        self.thrStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        

        self.rgbLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])
        
        self.thrLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])
        self.thrLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        
        self.rgbLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])

        self.bridge = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[4]*2,out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1)

            ])
        
        self.fuse1 = FusionBlock(channel=self.n_features[0],height=128//2,width=256//2)
        self.fuse2 = FusionBlock(channel=self.n_features[1],height=128//4,width=256//4)
        self.fuse3 = FusionBlock(channel=self.n_features[2],height=128//8,width=256//8)
        self.fuse4 = FusionBlock(channel=self.n_features[3],height=128//16,width=256//16)
        
        
        self.dLayer4 = ResConvBlock(self.n_features[4]+self.n_features[3]*2,self.n_features[3])
        self.dLayer3 = ResConvBlock(self.n_features[3]+self.n_features[2]*2,self.n_features[2])
        self.dLayer2 = ResConvBlock(self.n_features[2]+self.n_features[1]*2,self.n_features[1])
        self.dLayer1 = ResConvBlock(self.n_features[1]+self.n_features[0]*2,self.n_features[0])
        
        self.out = torch.nn.Conv2d(self.n_features[0],n_class,1)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                fan_out = m.weight.size(0)
                init_range = 1.0 / math.sqrt(fan_out)
                nn.init.uniform_(m.weight, -init_range, init_range)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        
    def forward(self, rgb, thr):
        thrStem = self.thrStem(thr)
        rgbStem = self.rgbStem(rgb)  # 256 
        
        
        thr = self.downsample(thrStem)
        rgb = self.downsample(rgbStem) # 128
        thr,rgb = self.fuse1(thr,rgb)
        
        thrLayer1 = self.thrLayer1(thr)
        rgbLayer1 = self.rgbLayer1(rgb)
        
        
        thr = self.downsample(thrLayer1)
        rgb = self.downsample(rgbLayer1) # 64 
#         thr,rgb = self.fuse2(thr,rgb)
        
        thrLayer2 = self.thrLayer2(thr)
        rgbLayer2 = self.rgbLayer2(rgb)
        
        
        thr = self.downsample(thrLayer2)
        rgb = self.downsample(rgbLayer2) # 32 
#         thr,rgb = self.fuse3(thr,rgb)
        
        thrLayer3 = self.thrLayer3(thr)
        rgbLayer3 = self.rgbLayer3(rgb)
        
        
        thr = self.downsample(thrLayer3)
        rgb = self.downsample(rgbLayer3) # 16
        thr,rgb = self.fuse4(thr,rgb)
        
        thrLayer4 = self.thrLayer4(thr)
        rgbLayer4 = self.rgbLayer4(rgb)

        sumLayer4 = self.bridge(torch.cat([rgbLayer4,thrLayer4],dim=1))
        
        
        x = self.upsample(sumLayer4)
        x = torch.cat([x,rgbLayer3,thrLayer3],dim=1)
        x = self.dLayer4(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer2,thrLayer2],dim=1)
        x = self.dLayer3(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer1,thrLayer1],dim=1)
        x = self.dLayer2(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbStem,thrStem],dim=1)
        x = self.dLayer1(x)
        
        out = self.out(x)
        
        return out
        
model = MBConvIntertwinedUNet(n_class=1)



In [20]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_class = 1

model = MBConvIntertwinedUNet(n_class=1).to(device)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=50, gamma=0.5)

model,history = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=100)

cuda:0
Epoch 0/99
----------
LR 0.0001
train: bce: 0.481730, dice: 0.351965, loss: 0.416847
val: bce: 0.319396, dice: 0.322956, loss: 0.321176
saving best model
train iou: 0.679970956509696 dice: 0.7550505503587335
valid iou: 0.7475103916294427 dice: 0.8077788930810209
10m 4s
Epoch 1/99
----------
LR 0.0001
train: bce: 0.340880, dice: 0.327447, loss: 0.334164
val: bce: 0.273585, dice: 0.318869, loss: 0.296227
saving best model
train iou: 0.7388873415399524 dice: 0.7947793824108303
valid iou: 0.7666601764787998 dice: 0.8138617042670977
10m 16s
Epoch 2/99
----------
LR 0.0001
train: bce: 0.312509, dice: 0.318184, loss: 0.315346
val: bce: 0.288868, dice: 0.308916, loss: 0.298892
train iou: 0.7577693506662201 dice: 0.8081780746129341
valid iou: 0.7793115439749652 dice: 0.8399427531660089
10m 3s
Epoch 3/99
----------
LR 0.0001
train: bce: 0.297060, dice: 0.312798, loss: 0.304929
val: bce: 0.259382, dice: 0.309399, loss: 0.284391
saving best model
train iou: 0.7649706962388692 dice: 0.816710

train: bce: 0.132041, dice: 0.101577, loss: 0.116809
val: bce: 0.442155, dice: 0.148124, loss: 0.295139
train iou: 0.8936193018098143 dice: 0.9255333497523482
valid iou: 0.824336309632786 dice: 0.8875238158198566
9m 11s
Epoch 33/99
----------
LR 0.0001
train: bce: 0.130962, dice: 0.099330, loss: 0.115146
val: bce: 0.332676, dice: 0.167570, loss: 0.250123
train iou: 0.893337002522857 dice: 0.9257699317630549
valid iou: 0.8240737051547885 dice: 0.8841244743109675
9m 11s
Epoch 34/99
----------
LR 0.0001
train: bce: 0.120536, dice: 0.091930, loss: 0.106233
val: bce: 0.399004, dice: 0.203470, loss: 0.301237
train iou: 0.9012955658530295 dice: 0.9309250246910088
valid iou: 0.7971670546963914 dice: 0.856463605652709
9m 11s
Epoch 35/99
----------
LR 0.0001
train: bce: 0.125931, dice: 0.098538, loss: 0.112234
val: bce: 0.360505, dice: 0.150832, loss: 0.255668
train iou: 0.8955979349767893 dice: 0.9272898503846722
valid iou: 0.826937439977731 dice: 0.887914968383682
9m 11s
Epoch 36/99
----------

train: bce: 0.049156, dice: 0.034997, loss: 0.042077
val: bce: 0.506923, dice: 0.133244, loss: 0.320083
train iou: 0.9554567653744175 dice: 0.970342475372062
valid iou: 0.8341674566264077 dice: 0.8999891743203594
9m 9s
Epoch 66/99
----------
LR 5e-05
train: bce: 0.045466, dice: 0.034750, loss: 0.040108
val: bce: 0.549332, dice: 0.134687, loss: 0.342009
train iou: 0.9540554754993074 dice: 0.9680505173490686
valid iou: 0.8257845557869878 dice: 0.8929842450911447
9m 9s
Epoch 67/99
----------
LR 5e-05
train: bce: 0.050621, dice: 0.036794, loss: 0.043708
val: bce: 0.473352, dice: 0.134561, loss: 0.303956
train iou: 0.9540013904297338 dice: 0.9684971025976038
valid iou: 0.8344472272791883 dice: 0.9002361879585741
9m 9s
Epoch 68/99
----------
LR 5e-05
train: bce: 0.048564, dice: 0.033958, loss: 0.041261
val: bce: 0.515198, dice: 0.128759, loss: 0.321978
train iou: 0.9559449676213235 dice: 0.9702740207814041
valid iou: 0.8398213849510162 dice: 0.9029846848130709
9m 8s
Epoch 69/99
----------
LR

train: bce: 0.034441, dice: 0.027434, loss: 0.030937
val: bce: 0.582996, dice: 0.126498, loss: 0.354747
train iou: 0.9670032959138117 dice: 0.9780617939550398
valid iou: 0.8358569507276188 dice: 0.8995206826574134
9m 8s
Epoch 99/99
----------
LR 2.5e-05
train: bce: 0.029629, dice: 0.022279, loss: 0.025954
val: bce: 0.537626, dice: 0.127754, loss: 0.332690
train iou: 0.9719060815198717 dice: 0.9813194401090152
valid iou: 0.8390502380350604 dice: 0.9049255331595317
9m 8s
Best val loss: 0.227389


In [21]:
model = model.cpu()

torch.save(model.state_dict(),'./MBConvNetFuse14_gamma05.pt')

import pickle
with open('./model_history_files/MBConvNetFuse14_gamma05.bin','wb') as f:
    pickle.dump(history,f)

In [24]:
model = model.cuda()
torch.cuda.empty_cache()

test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)

test: bce: 0.550094, dice: 0.125442, loss: 0.337768
test_model ## iou: 0.8326234202908646 dice: 0.9004275141894806
test: bce: 0.550094, dice: 0.125442, loss: 0.337768
test_model ## iou: 0.8314768484092652 dice: 0.8983017174251459
test: bce: 0.550094, dice: 0.125442, loss: 0.337768
test_model ## iou: 0.8378059226306489 dice: 0.9010959511462802
test: bce: 0.550094, dice: 0.125442, loss: 0.337768
test_model ## iou: 0.8378560132520788 dice: 0.9021364363613028
test: bce: 0.550094, dice: 0.125442, loss: 0.337768
test_model ## iou: 0.8349548746154926 dice: 0.9014897900863406


2 4 fuse, stpe size 50, gamma 0.5

In [25]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
import math



class Swish(nn.Module):

    def __init__(self, *args, **kwargs):
        super(Swish, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)


class ConvBNReLU(nn.Sequential):

    def __init__(self, in_planes, out_planes, kernel_size, stride=1, groups=1):
        padding = self._get_padding(kernel_size, stride)
        super(ConvBNReLU, self).__init__(
            nn.ZeroPad2d(padding),
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding=0, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes),
            Swish(),
        )

    def _get_padding(self, kernel_size, stride):
        p = max(kernel_size - stride, 0)
        return [p // 2, p - p // 2, p // 2, p - p // 2]

class SqueezeExcitation(nn.Module):

    def __init__(self, in_planes, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_planes, reduced_dim, 1),
            Swish(),
            nn.Conv2d(reduced_dim, in_planes, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)

class MBConvBlock(nn.Module):

    def __init__(self,
                 in_planes,
                 out_planes,
                 expand_ratio,
                 kernel_size,
                 stride,
                 reduction_ratio=4,
                 drop_connect_rate=0.2):
        super(MBConvBlock, self).__init__()
        self.drop_connect_rate = drop_connect_rate
        self.use_residual = in_planes == out_planes and stride == 1
        assert stride in [1, 2]
        assert kernel_size in [3, 5]

        hidden_dim = in_planes * expand_ratio
        reduced_dim = max(1, int(in_planes / reduction_ratio))

        layers = []
        # pw
        if in_planes != hidden_dim:
            layers += [ConvBNReLU(in_planes, hidden_dim, 1)]

        layers += [
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, kernel_size, stride=stride, groups=hidden_dim),
            # se
            SqueezeExcitation(hidden_dim, reduced_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, out_planes, 1, bias=False),
            nn.BatchNorm2d(out_planes),
        ]

        self.conv = nn.Sequential(*layers)

    def _drop_connect(self, x):
        if not self.training:
            return x
        keep_prob = 1.0 - self.drop_connect_rate
        batch_size = x.size(0)
        random_tensor = keep_prob
        random_tensor += torch.rand(batch_size, 1, 1, 1, device=x.device)
        binary_tensor = random_tensor.floor()
        return x.div(keep_prob) * binary_tensor

    def forward(self, x):
        if self.use_residual:
            return x + self._drop_connect(self.conv(x))
        else:
            return self.conv(x)
        


class ResConvBlock(torch.nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.layer = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
            torch.nn.Conv2d(in_channels=out_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
        ])
        self.shortcut = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                           out_channels=out_channels,
                           kernel_size=(1,1),stride=1),
            torch.nn.BatchNorm2d(out_channels)
        ])
    def forward(self,x):
        shortcut = self.shortcut(x)
        x = self.layer(x)
        return x + shortcut

    
class FusionBlock(torch.nn.Module):
    def __init__(self,channel,height,width):
        super(FusionBlock, self).__init__()
        self.weightT = torch.nn.Parameter(torch.rand(size=(channel,height,width),requires_grad = True))
        self.weightR = torch.nn.Parameter(torch.rand(size=(channel,height,width),requires_grad = True))
        
    def forward(self,thr,rgb):
        thrAlpha = torch.sigmoid(self.weightT)
        thrBeta = 1-thrAlpha
        
        rgbAlpha = torch.sigmoid(self.weightR)
        rgbBeta = 1-rgbAlpha
        
        thr_ = thrAlpha*thr + thrBeta*rgb
        rgb_ = rgbAlpha*rgb + rgbBeta*thr
        
        return thr_,rgb_

class MBConvIntertwinedUNet(torch.nn.Module):
    def __init__(self, n_class):
        super().__init__()
        self.n_features = [64,256,256,256,256]
        self.downsample = torch.nn.MaxPool2d(2)
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        

        self.rgbStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        self.thrStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        

        self.rgbLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])
        
        self.thrLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])
        self.thrLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        
        self.rgbLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])

        self.bridge = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[4]*2,out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1)

            ])
        
        self.fuse1 = FusionBlock(channel=self.n_features[0],height=128//2,width=256//2)
        self.fuse2 = FusionBlock(channel=self.n_features[1],height=128//4,width=256//4)
        self.fuse3 = FusionBlock(channel=self.n_features[2],height=128//8,width=256//8)
        self.fuse4 = FusionBlock(channel=self.n_features[3],height=128//16,width=256//16)
        
        
        self.dLayer4 = ResConvBlock(self.n_features[4]+self.n_features[3]*2,self.n_features[3])
        self.dLayer3 = ResConvBlock(self.n_features[3]+self.n_features[2]*2,self.n_features[2])
        self.dLayer2 = ResConvBlock(self.n_features[2]+self.n_features[1]*2,self.n_features[1])
        self.dLayer1 = ResConvBlock(self.n_features[1]+self.n_features[0]*2,self.n_features[0])
        
        self.out = torch.nn.Conv2d(self.n_features[0],n_class,1)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                fan_out = m.weight.size(0)
                init_range = 1.0 / math.sqrt(fan_out)
                nn.init.uniform_(m.weight, -init_range, init_range)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        
    def forward(self, rgb, thr):
        thrStem = self.thrStem(thr)
        rgbStem = self.rgbStem(rgb)  # 256 
        
        
        thr = self.downsample(thrStem)
        rgb = self.downsample(rgbStem) # 128
#         thr,rgb = self.fuse1(thr,rgb)
        
        thrLayer1 = self.thrLayer1(thr)
        rgbLayer1 = self.rgbLayer1(rgb)
        
        
        thr = self.downsample(thrLayer1)
        rgb = self.downsample(rgbLayer1) # 64 
        thr,rgb = self.fuse2(thr,rgb)
        
        thrLayer2 = self.thrLayer2(thr)
        rgbLayer2 = self.rgbLayer2(rgb)
        
        
        thr = self.downsample(thrLayer2)
        rgb = self.downsample(rgbLayer2) # 32 
#         thr,rgb = self.fuse3(thr,rgb)
        
        thrLayer3 = self.thrLayer3(thr)
        rgbLayer3 = self.rgbLayer3(rgb)
        
        
        thr = self.downsample(thrLayer3)
        rgb = self.downsample(rgbLayer3) # 16
        thr,rgb = self.fuse4(thr,rgb)
        
        thrLayer4 = self.thrLayer4(thr)
        rgbLayer4 = self.rgbLayer4(rgb)

        sumLayer4 = self.bridge(torch.cat([rgbLayer4,thrLayer4],dim=1))
        
        
        x = self.upsample(sumLayer4)
        x = torch.cat([x,rgbLayer3,thrLayer3],dim=1)
        x = self.dLayer4(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer2,thrLayer2],dim=1)
        x = self.dLayer3(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer1,thrLayer1],dim=1)
        x = self.dLayer2(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbStem,thrStem],dim=1)
        x = self.dLayer1(x)
        
        out = self.out(x)
        
        return out
        
model = MBConvIntertwinedUNet(n_class=1)



In [26]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_class = 1

model = MBConvIntertwinedUNet(n_class=1).to(device)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=50, gamma=0.5)

model,history = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=100)

cuda:0
Epoch 0/99
----------
LR 0.0001
train: bce: 0.484309, dice: 0.350970, loss: 0.417639
val: bce: 0.299154, dice: 0.335440, loss: 0.317297
saving best model
train iou: 0.6792883575532426 dice: 0.7583499754478676
valid iou: 0.750867759570133 dice: 0.8052290624768215
9m 59s
Epoch 1/99
----------
LR 0.0001
train: bce: 0.345874, dice: 0.327209, loss: 0.336541
val: bce: 0.279821, dice: 0.320967, loss: 0.300394
saving best model
train iou: 0.7344933938680577 dice: 0.7915311846119796
valid iou: 0.7780610226864426 dice: 0.8247044291271187
10m 8s
Epoch 2/99
----------
LR 0.0001
train: bce: 0.313111, dice: 0.316516, loss: 0.314814
val: bce: 0.285256, dice: 0.302873, loss: 0.294064
saving best model
train iou: 0.754542611439074 dice: 0.8071961520464365
valid iou: 0.7750335762939546 dice: 0.8241876430142239
10m 8s
Epoch 3/99
----------
LR 0.0001
train: bce: 0.294680, dice: 0.311355, loss: 0.303018
val: bce: 0.324287, dice: 0.293765, loss: 0.309026
train iou: 0.7651237640540897 dice: 0.81834183

train: bce: 0.143616, dice: 0.111854, loss: 0.127735
val: bce: 0.428533, dice: 0.164278, loss: 0.296406
train iou: 0.8839790943541886 dice: 0.9184351279293567
valid iou: 0.816236584312341 dice: 0.8792674845087528
10m 52s
Epoch 33/99
----------
LR 0.0001
train: bce: 0.130254, dice: 0.099142, loss: 0.114698
val: bce: 0.384274, dice: 0.150541, loss: 0.267408
train iou: 0.8935797902988228 dice: 0.9255706333905621
valid iou: 0.8345547273945237 dice: 0.890665306946183
10m 50s
Epoch 34/99
----------
LR 0.0001
train: bce: 0.130070, dice: 0.099524, loss: 0.114797
val: bce: 0.426109, dice: 0.160560, loss: 0.293335
train iou: 0.8953203345571651 dice: 0.9263827943801656
valid iou: 0.8126310263445441 dice: 0.8786775142454879
10m 53s
Epoch 35/99
----------
LR 0.0001
train: bce: 0.122912, dice: 0.095391, loss: 0.109152
val: bce: 0.561952, dice: 0.153314, loss: 0.357633
train iou: 0.8993674174359865 dice: 0.9295787964715224
valid iou: 0.8096463047023644 dice: 0.8766200804986828
10m 49s
Epoch 36/99
---

train: bce: 0.049969, dice: 0.035727, loss: 0.042848
val: bce: 0.465845, dice: 0.131749, loss: 0.298797
train iou: 0.9535027761161241 dice: 0.9688009594156213
valid iou: 0.8403997836961472 dice: 0.9042381575348949
10m 27s
Epoch 66/99
----------
LR 5e-05
train: bce: 0.047899, dice: 0.033413, loss: 0.040656
val: bce: 0.488940, dice: 0.136290, loss: 0.312615
train iou: 0.9558259133193444 dice: 0.9706304652617926
valid iou: 0.8337920037993767 dice: 0.9000989266184114
10m 30s
Epoch 67/99
----------
LR 5e-05
train: bce: 0.049586, dice: 0.036018, loss: 0.042802
val: bce: 0.484375, dice: 0.145298, loss: 0.314836
train iou: 0.955437707045946 dice: 0.970003020044135
valid iou: 0.8276338457976926 dice: 0.8936684740522722
10m 27s
Epoch 68/99
----------
LR 5e-05
train: bce: 0.046301, dice: 0.034382, loss: 0.040342
val: bce: 0.547007, dice: 0.139799, loss: 0.343403
train iou: 0.9562944587985739 dice: 0.9706305737537645
valid iou: 0.8297214402840223 dice: 0.8942837617615079
10m 29s
Epoch 69/99
------

train: bce: 0.032248, dice: 0.025178, loss: 0.028713
val: bce: 0.620875, dice: 0.122689, loss: 0.371782
train iou: 0.9668798928273397 dice: 0.9772664220343621
valid iou: 0.8348878506572188 dice: 0.9010979652258186
9m 29s
Epoch 99/99
----------
LR 2.5e-05
train: bce: 0.031686, dice: 0.023857, loss: 0.027771
val: bce: 0.570179, dice: 0.123741, loss: 0.346960
train iou: 0.9709330420903106 dice: 0.9807950841375055
valid iou: 0.8357300045279537 dice: 0.9011507631894111
9m 29s
Best val loss: 0.235006


In [27]:
model = model.cpu()

torch.save(model.state_dict(),'./MBConvNetFuse24_gamma05.pt')

import pickle
with open('./model_history_files/MBConvNetFuse24_gamma05.bin','wb') as f:
    pickle.dump(history,f)

In [28]:
model = model.cpu()

torch.save(model.state_dict(),'./MBConvNetFuse24_gamma05.pt')

import pickle
with open('./model_history_files/MBConvNetFuse24_gamma05.bin','wb') as f:
    pickle.dump(history,f)

model = model.cuda()
torch.cuda.empty_cache()

test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)

test: bce: 0.702117, dice: 0.132232, loss: 0.417174
test_model ## iou: 0.8311663817840172 dice: 0.8995004948401403
test: bce: 0.702117, dice: 0.132232, loss: 0.417174
test_model ## iou: 0.8211935285631459 dice: 0.890128890821761
test: bce: 0.702117, dice: 0.132232, loss: 0.417174
test_model ## iou: 0.8269980212083202 dice: 0.8946896406870941
test: bce: 0.702117, dice: 0.132232, loss: 0.417174
test_model ## iou: 0.8260784049835672 dice: 0.8944746769594935
test: bce: 0.702117, dice: 0.132232, loss: 0.417174
test_model ## iou: 0.8260488160753048 dice: 0.8930994955684415


no fuse, stpe size 50, gamma 0.5

In [10]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
import math



class Swish(nn.Module):

    def __init__(self, *args, **kwargs):
        super(Swish, self).__init__()

    def forward(self, x):
        return x * torch.sigmoid(x)


class ConvBNReLU(nn.Sequential):

    def __init__(self, in_planes, out_planes, kernel_size, stride=1, groups=1):
        padding = self._get_padding(kernel_size, stride)
        super(ConvBNReLU, self).__init__(
            nn.ZeroPad2d(padding),
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding=0, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes),
            Swish(),
        )

    def _get_padding(self, kernel_size, stride):
        p = max(kernel_size - stride, 0)
        return [p // 2, p - p // 2, p // 2, p - p // 2]

class SqueezeExcitation(nn.Module):

    def __init__(self, in_planes, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_planes, reduced_dim, 1),
            Swish(),
            nn.Conv2d(reduced_dim, in_planes, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)

class MBConvBlock(nn.Module):

    def __init__(self,
                 in_planes,
                 out_planes,
                 expand_ratio,
                 kernel_size,
                 stride,
                 reduction_ratio=4,
                 drop_connect_rate=0.2):
        super(MBConvBlock, self).__init__()
        self.drop_connect_rate = drop_connect_rate
        self.use_residual = in_planes == out_planes and stride == 1
        assert stride in [1, 2]
        assert kernel_size in [3, 5]

        hidden_dim = in_planes * expand_ratio
        reduced_dim = max(1, int(in_planes / reduction_ratio))

        layers = []
        # pw
        if in_planes != hidden_dim:
            layers += [ConvBNReLU(in_planes, hidden_dim, 1)]

        layers += [
            # dw
            ConvBNReLU(hidden_dim, hidden_dim, kernel_size, stride=stride, groups=hidden_dim),
            # se
            SqueezeExcitation(hidden_dim, reduced_dim),
            # pw-linear
            nn.Conv2d(hidden_dim, out_planes, 1, bias=False),
            nn.BatchNorm2d(out_planes),
        ]

        self.conv = nn.Sequential(*layers)

    def _drop_connect(self, x):
        if not self.training:
            return x
        keep_prob = 1.0 - self.drop_connect_rate
        batch_size = x.size(0)
        random_tensor = keep_prob
        random_tensor += torch.rand(batch_size, 1, 1, 1, device=x.device)
        binary_tensor = random_tensor.floor()
        return x.div(keep_prob) * binary_tensor

    def forward(self, x):
        if self.use_residual:
            return x + self._drop_connect(self.conv(x))
        else:
            return self.conv(x)
        


class ResConvBlock(torch.nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.layer = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
            torch.nn.Conv2d(in_channels=out_channels,
                            out_channels=out_channels,
                            kernel_size=(3,3),stride=1,padding=1
                                 ),
            torch.nn.BatchNorm2d(out_channels),
            Swish(),
        ])
        self.shortcut = torch.nn.Sequential(*[
            torch.nn.Conv2d(in_channels=in_channels,
                           out_channels=out_channels,
                           kernel_size=(1,1),stride=1),
            torch.nn.BatchNorm2d(out_channels)
        ])
    def forward(self,x):
        shortcut = self.shortcut(x)
        x = self.layer(x)
        return x + shortcut

    
class FusionBlock(torch.nn.Module):
    def __init__(self,channel,height,width):
        super(FusionBlock, self).__init__()
        self.weightT = torch.nn.Parameter(torch.rand(size=(channel,height,width),requires_grad = True))
        self.weightR = torch.nn.Parameter(torch.rand(size=(channel,height,width),requires_grad = True))
        
    def forward(self,thr,rgb):
        thrAlpha = torch.sigmoid(self.weightT)
        thrBeta = 1-thrAlpha
        
        rgbAlpha = torch.sigmoid(self.weightR)
        rgbBeta = 1-rgbAlpha
        
        thr_ = thrAlpha*thr + thrBeta*rgb
        rgb_ = rgbAlpha*rgb + rgbBeta*thr
        
        return thr_,rgb_

class MBConvIntertwinedUNet(torch.nn.Module):
    def __init__(self, n_class):
        super().__init__()
        self.n_features = [64,256,256,256,256]
        self.downsample = torch.nn.MaxPool2d(2)
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        

        self.rgbStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        self.thrStem = MBConvBlock(in_planes=3,out_planes=self.n_features[0], expand_ratio=1,kernel_size=3,stride=1)
        

        self.rgbLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])
        
        self.thrLayer1 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[0],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[1], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])
        self.thrLayer2 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[1],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[2], expand_ratio=6,kernel_size=3,stride=1),
            ])

        self.rgbLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer3 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[2],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[3], expand_ratio=6,kernel_size=5,stride=1),
            ])
        
        self.rgbLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])
        self.thrLayer4 = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[3],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            MBConvBlock(in_planes=self.n_features[4],out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1),
            ])

        self.bridge = torch.nn.Sequential(*[
            MBConvBlock(in_planes=self.n_features[4]*2,out_planes=self.n_features[4], expand_ratio=6,kernel_size=5,stride=1)

            ])
        
#         self.fuse1 = FusionBlock(channel=self.n_features[0],height=128//2,width=256//2)
#         self.fuse2 = FusionBlock(channel=self.n_features[1],height=128//4,width=256//4)
#         self.fuse3 = FusionBlock(channel=self.n_features[2],height=128//8,width=256//8)
#         self.fuse4 = FusionBlock(channel=self.n_features[3],height=128//16,width=256//16)
        
        
        self.dLayer4 = ResConvBlock(self.n_features[4]+self.n_features[3]*2,self.n_features[3])
        self.dLayer3 = ResConvBlock(self.n_features[3]+self.n_features[2]*2,self.n_features[2])
        self.dLayer2 = ResConvBlock(self.n_features[2]+self.n_features[1]*2,self.n_features[1])
        self.dLayer1 = ResConvBlock(self.n_features[1]+self.n_features[0]*2,self.n_features[0])
        
        self.out = torch.nn.Conv2d(self.n_features[0],n_class,1)
        
        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                fan_out = m.weight.size(0)
                init_range = 1.0 / math.sqrt(fan_out)
                nn.init.uniform_(m.weight, -init_range, init_range)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
        
    def forward(self, rgb, thr):
        thrStem = self.thrStem(thr)
        rgbStem = self.rgbStem(rgb)  # 256 
        
        
        thr = self.downsample(thrStem)
        rgb = self.downsample(rgbStem) # 128
#         thr,rgb = self.fuse1(thr,rgb)
        
        thrLayer1 = self.thrLayer1(thr)
        rgbLayer1 = self.rgbLayer1(rgb)
        
        
        thr = self.downsample(thrLayer1)
        rgb = self.downsample(rgbLayer1) # 64 
#         thr,rgb = self.fuse2(thr,rgb)
        
        thrLayer2 = self.thrLayer2(thr)
        rgbLayer2 = self.rgbLayer2(rgb)
        
        
        thr = self.downsample(thrLayer2)
        rgb = self.downsample(rgbLayer2) # 32 
#         thr,rgb = self.fuse3(thr,rgb)
        
        thrLayer3 = self.thrLayer3(thr)
        rgbLayer3 = self.rgbLayer3(rgb)
        
        
        thr = self.downsample(thrLayer3)
        rgb = self.downsample(rgbLayer3) # 16
#         thr,rgb = self.fuse4(thr,rgb)
        
        thrLayer4 = self.thrLayer4(thr)
        rgbLayer4 = self.rgbLayer4(rgb)

        sumLayer4 = self.bridge(torch.cat([rgbLayer4,thrLayer4],dim=1))
        
        
        x = self.upsample(sumLayer4)
        x = torch.cat([x,rgbLayer3,thrLayer3],dim=1)
        x = self.dLayer4(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer2,thrLayer2],dim=1)
        x = self.dLayer3(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbLayer1,thrLayer1],dim=1)
        x = self.dLayer2(x)
        
        x = self.upsample(x)
        x = torch.cat([x,rgbStem,thrStem],dim=1)
        x = self.dLayer1(x)
        
        out = self.out(x)
        
        return out
        
model = MBConvIntertwinedUNet(n_class=1)



In [11]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_class = 1

model = MBConvIntertwinedUNet(n_class=1).to(device)

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model.parameters(), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=50, gamma=0.5)

model,history = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=100)

cuda:0
Epoch 0/99
----------
LR 0.0001


/compuworks/anaconda3/envs/py36torch/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/compuworks/anaconda3/envs/py36torch/lib/python3.6/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train: bce: 0.511480, dice: 0.349917, loss: 0.430699
val: bce: 0.342561, dice: 0.316964, loss: 0.329762
saving best model
train iou: 0.6770675296441976 dice: 0.7566047530557313
valid iou: 0.7498981094652426 dice: 0.8146244632146633
9m 2s
Epoch 1/99
----------
LR 0.0001
train: bce: 0.348621, dice: 0.325440, loss: 0.337031
val: bce: 0.280403, dice: 0.332861, loss: 0.306632
saving best model
train iou: 0.7356091606979304 dice: 0.7935264543794659
valid iou: 0.7670351777744713 dice: 0.798102460807729
9m 8s
Epoch 2/99
----------
LR 0.0001
train: bce: 0.317062, dice: 0.316678, loss: 0.316870
val: bce: 0.289336, dice: 0.303303, loss: 0.296320
saving best model
train iou: 0.7553365027144799 dice: 0.8089836859960737
valid iou: 0.7778940584475253 dice: 0.8405006888422326
9m 8s
Epoch 3/99
----------
LR 0.0001
train: bce: 0.303556, dice: 0.311473, loss: 0.307514
val: bce: 0.263516, dice: 0.310906, loss: 0.287211
saving best model
train iou: 0.7630876572890166 dice: 0.8150791601731173
valid iou: 0.7

train: bce: 0.128975, dice: 0.099048, loss: 0.114012
val: bce: 0.371992, dice: 0.156690, loss: 0.264341
train iou: 0.8938419980569657 dice: 0.9256063437487495
valid iou: 0.8278302295181725 dice: 0.8858178815532355
9m 10s
Epoch 33/99
----------
LR 0.0001
train: bce: 0.123167, dice: 0.094072, loss: 0.108619
val: bce: 0.349249, dice: 0.142829, loss: 0.246039
train iou: 0.8989669505491971 dice: 0.9293990578052065
valid iou: 0.8378157173344581 dice: 0.8978984419281373
9m 10s
Epoch 34/99
----------
LR 0.0001
train: bce: 0.120411, dice: 0.093657, loss: 0.107034
val: bce: 0.319345, dice: 0.162320, loss: 0.240833
train iou: 0.8977326839084766 dice: 0.928630341916546
valid iou: 0.820740751622443 dice: 0.8779113427156214
9m 10s
Epoch 35/99
----------
LR 0.0001
train: bce: 0.120760, dice: 0.092956, loss: 0.106858
val: bce: 0.390788, dice: 0.134985, loss: 0.262886
train iou: 0.9021701305568786 dice: 0.9319492917334473
valid iou: 0.8369987976359181 dice: 0.8969000700475296
9m 10s
Epoch 36/99
-------

train: bce: 0.045304, dice: 0.031050, loss: 0.038177
val: bce: 0.449412, dice: 0.130947, loss: 0.290179
train iou: 0.9579942725943749 dice: 0.9715570898918339
valid iou: 0.840643307166798 dice: 0.9039990929674913
9m 10s
Epoch 66/99
----------
LR 5e-05
train: bce: 0.044167, dice: 0.031819, loss: 0.037993
val: bce: 0.448896, dice: 0.135744, loss: 0.292320
train iou: 0.9600536742507643 dice: 0.9734663870919495
valid iou: 0.8398118727237238 dice: 0.9026085882403521
9m 10s
Epoch 67/99
----------
LR 5e-05
train: bce: 0.043137, dice: 0.031331, loss: 0.037234
val: bce: 0.562965, dice: 0.133949, loss: 0.348457
train iou: 0.9591109345636788 dice: 0.9724959110928583
valid iou: 0.8313805084737773 dice: 0.8984073403174199
9m 9s
Epoch 68/99
----------
LR 5e-05
train: bce: 0.043475, dice: 0.032549, loss: 0.038012
val: bce: 0.427743, dice: 0.129637, loss: 0.278690
train iou: 0.9581415138237326 dice: 0.9716887801198473
valid iou: 0.8442121181463809 dice: 0.9066695639055464
9m 10s
Epoch 69/99
----------

train: bce: 0.034479, dice: 0.024103, loss: 0.029291
val: bce: 0.499932, dice: 0.124555, loss: 0.312243
train iou: 0.9701498072937561 dice: 0.9804300045602742
valid iou: 0.8425448407461622 dice: 0.9031516723229356
9m 10s
Epoch 99/99
----------
LR 2.5e-05
train: bce: 0.027498, dice: 0.022204, loss: 0.024851
val: bce: 0.478915, dice: 0.127005, loss: 0.302960
train iou: 0.9731226061326211 dice: 0.9817991721805682
valid iou: 0.8411278425389057 dice: 0.9062174886532293
9m 10s
Best val loss: 0.221994


In [12]:
model = model.cpu()

torch.save(model.state_dict(),'./MBConvNetNoFuse_gamma05.pt')

import pickle
with open('./model_history_files/MBConvNetNoFuse_gamma05.bin','wb') as f:
    pickle.dump(history,f)

model = model.cuda()
torch.cuda.empty_cache()

test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)
test_model(model,testLoader)

test: bce: 0.487138, dice: 0.125887, loss: 0.306513
test_model ## iou: 0.8373745174180958 dice: 0.9024144405367853
test: bce: 0.487138, dice: 0.125887, loss: 0.306513
test_model ## iou: 0.8376229136973387 dice: 0.9024798519008531
test: bce: 0.487138, dice: 0.125887, loss: 0.306513
test_model ## iou: 0.8372949152841047 dice: 0.9024791150926518
test: bce: 0.487138, dice: 0.125887, loss: 0.306513
test_model ## iou: 0.8361186428147307 dice: 0.9021101342178276
test: bce: 0.487138, dice: 0.125887, loss: 0.306513
test_model ## iou: 0.8385340631478635 dice: 0.9025532572200372
